<a href="https://colab.research.google.com/github/DelMashiry-dev/DelMashiry-dev/blob/main/MONDAY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Understanding

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

# Mount Google Drive and load data
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/Medical_Resource_Prediction/owid-covid-data.csv'
df = pd.read_csv(data_path, parse_dates=['date'])

# Initial exploration
print(f"Dataset shape: {df.shape}")
print("\nData types:\n", df.dtypes.value_counts())
print("\nMissing values percentage:\n", df.isnull().mean().sort_values(ascending=False).head(20))

In [ ]:
# Medical Resource Demand Prediction System - Complete Google Colab Implementation
# ============================================================================

# PART 1: INSTALLATION AND IMPORTS
# ============================================================================

# Install required packages
!pip install plotly dash jupyter-dash scikit-learn xgboost tensorflow pandas numpy matplotlib seaborn requests

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import dash
from dash import dcc, html, Input, Output, dash_table
import jupyter_dash
from datetime import datetime, timedelta
import warnings
import os
import requests
warnings.filterwarnings('ignore')

# Machine Learning imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import VotingRegressor

# Setup Google Colab environment
def setup_google_colab():
    """Setup function specifically for Google Colab users"""
    print("🔧 Setting up Google Colab environment...")

    # Mount Google Drive
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("✅ Google Drive mounted successfully")
    except ImportError:
        print("⚠️  Not running in Google Colab environment")
    except Exception as e:
        print(f"⚠️  Drive mounting failed: {e}")

    # Create necessary directories
    os.makedirs('/content/drive/MyDrive/Medical_Resource_Prediction', exist_ok=True)
    print("✅ Project directories created")

    # Download sample data
    data_url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
    data_path = '/content/drive/MyDrive/Medical_Resource_Prediction/owid-covid-data.csv'

    if not os.path.exists(data_path):
        print("📥 Downloading COVID-19 dataset...")
        try:
            response = requests.get(data_url)
            response.raise_for_status()
            with open(data_path, 'wb') as f:
                f.write(response.content)
            print("✅ Dataset downloaded successfully")
        except Exception as e:
            print(f"❌ Failed to download dataset: {e}")
            # Try alternative path
            data_path = '/content/owid-covid-data.csv'
            if not os.path.exists(data_path):
                response = requests.get(data_url)
                with open(data_path, 'wb') as f:
                    f.write(response.content)
                print(f"✅ Dataset saved to {data_path}")
    else:
        print("✅ Dataset already exists")

    return data_path

# PART 2: MEDICAL RESOURCE PREDICTOR CLASS
# ============================================================================

class MedicalResourcePredictor:
    def __init__(self, data_path):
        self.data_path = data_path
        self.data = None
        self.processed_data = None
        self.models = {}
        self.scalers = {}
        self.feature_importance = {}
        self.training_results = {}

    def load_data(self):
        """Load and initial examination of data"""
        print("📊 Loading COVID-19 data...")
        try:
            self.data = pd.read_csv(self.data_path)
            print(f"✅ Data loaded successfully. Shape: {self.data.shape}")
            print(f"📋 Columns: {list(self.data.columns[:10])}...")  # Show first 10 columns
            return self.data
        except Exception as e:
            print(f"❌ Error loading data: {e}")
            return None

    def preprocess_data(self):
        """Comprehensive data preprocessing and feature engineering"""
        print("🔄 Starting data preprocessing...")

        if self.data is None:
            print("❌ No data available for preprocessing")
            return None

        try:
            # Convert date column
            self.data['date'] = pd.to_datetime(self.data['date'])

            # Select relevant columns for medical resource prediction
            resource_columns = [
                'date', 'location', 'total_cases', 'new_cases', 'total_deaths',
                'new_deaths', 'total_tests', 'new_tests', 'population',
                'population_density', 'median_age', 'aged_65_older',
                'aged_70_older', 'gdp_per_capita', 'hospital_beds_per_thousand',
                'icu_patients', 'hosp_patients', 'weekly_icu_admissions',
                'weekly_hosp_admissions', 'total_vaccinations', 'people_vaccinated',
                'people_fully_vaccinated', 'total_boosters', 'stringency_index'
            ]

            # Filter columns that exist in the dataset
            available_columns = [col for col in resource_columns if col in self.data.columns]
            df = self.data[available_columns].copy()

            print(f"📋 Available columns: {len(available_columns)}/{len(resource_columns)}")

            # Focus on countries with sufficient data (at least 100 records)
            country_data_counts = df.groupby('location').size()
            countries_with_data = country_data_counts[country_data_counts >= 100].index
            df = df[df['location'].isin(countries_with_data)].copy()

            print(f"🌍 Countries with sufficient data: {len(countries_with_data)}")

            # Feature Engineering
            print("⚙️ Engineering features...")

            # Time-based features
            df['year'] = df['date'].dt.year
            df['month'] = df['date'].dt.month
            df['day_of_year'] = df['date'].dt.dayofyear
            df['week_of_year'] = df['date'].dt.isocalendar().week

            # Sort data for rolling calculations
            df = df.sort_values(['location', 'date'])

            # Rolling averages for trend analysis
            for col in ['new_cases', 'new_deaths', 'new_tests']:
                if col in df.columns:
                    df[f'{col}_7day_avg'] = df.groupby('location')[col].rolling(7, min_periods=1).mean().reset_index(0, drop=True)
                    df[f'{col}_14day_avg'] = df.groupby('location')[col].rolling(14, min_periods=1).mean().reset_index(0, drop=True)

            # Growth rates
            for col in ['total_cases', 'total_deaths']:
                if col in df.columns:
                    df[f'{col}_growth_rate'] = df.groupby('location')[col].pct_change().fillna(0)
                    # Cap extreme growth rates
                    df[f'{col}_growth_rate'] = df[f'{col}_growth_rate'].clip(-1, 5)

            # Case fatality rate
            if 'total_cases' in df.columns and 'total_deaths' in df.columns:
                df['case_fatality_rate'] = np.where(df['total_cases'] > 0,
                                                   df['total_deaths'] / df['total_cases'], 0)
                df['case_fatality_rate'] = df['case_fatality_rate'].clip(0, 0.2)  # Cap at 20%

            # Testing rate
            if 'total_tests' in df.columns and 'population' in df.columns:
                df['tests_per_capita'] = np.where(df['population'] > 0,
                                                 df['total_tests'] / df['population'], 0)
                df['tests_per_capita'] = df['tests_per_capita'].clip(0, 10)  # Cap at reasonable level

            # Vaccination rate
            if 'people_fully_vaccinated' in df.columns and 'population' in df.columns:
                df['vaccination_rate'] = np.where(df['population'] > 0,
                                                df['people_fully_vaccinated'] / df['population'], 0)
                df['vaccination_rate'] = df['vaccination_rate'].clip(0, 1.2)  # Cap at 120%

            # Create target variables (medical resource demands)
            print("🎯 Creating target variables...")

            # ICU demand prediction
            if 'icu_patients' in df.columns:
                df['icu_demand'] = df['icu_patients'].fillna(0)
            else:
                # Estimate ICU demand based on severe cases (5% of active cases)
                if 'total_cases' in df.columns and 'total_deaths' in df.columns:
                    # Estimate recovered cases (rough approximation)
                    df['estimated_recovered'] = df.groupby('location')['total_cases'].shift(14).fillna(0) * 0.95
                    df['estimated_active_cases'] = (df['total_cases'] - df['total_deaths'] - df['estimated_recovered']).clip(0)
                    df['icu_demand'] = df['estimated_active_cases'] * 0.05
                else:
                    df['icu_demand'] = df['new_cases'] * 0.1 if 'new_cases' in df.columns else 0

            # Hospital bed demand
            if 'hosp_patients' in df.columns:
                df['hospital_demand'] = df['hosp_patients'].fillna(0)
            else:
                # Estimate hospital demand (15% of active cases)
                if 'estimated_active_cases' in df.columns:
                    df['hospital_demand'] = df['estimated_active_cases'] * 0.15
                else:
                    df['hospital_demand'] = df['new_cases'] * 0.2 if 'new_cases' in df.columns else 0

            # Ventilator demand (80% of ICU patients)
            df['ventilator_demand'] = df['icu_demand'] * 0.8

            # PPE demand (based on healthcare workers and case load)
            if 'new_cases' in df.columns:
                df['ppe_demand'] = df['new_cases'] * 10  # 10 PPE units per case
            else:
                df['ppe_demand'] = 100  # Default minimal demand

            # Handle missing values
            numeric_columns = df.select_dtypes(include=[np.number]).columns
            for col in numeric_columns:
                df[col] = df[col].fillna(df[col].median())

            # Encode categorical variables
            if 'location' in df.columns:
                le = LabelEncoder()
                df['location_encoded'] = le.fit_transform(df['location'])
                self.location_encoder = le

            # Remove extreme outliers using IQR method for target variables
            for col in ['icu_demand', 'hospital_demand', 'ventilator_demand', 'ppe_demand']:
                Q1 = df[col].quantile(0.05)  # Use 5th and 95th percentiles instead of IQR
                Q3 = df[col].quantile(0.95)
                df[col] = df[col].clip(Q1, Q3)

            self.processed_data = df
            print(f"✅ Preprocessing complete. Final data shape: {df.shape}")
            print(f"📊 Date range: {df['date'].min()} to {df['date'].max()}")
            return df

        except Exception as e:
            print(f"❌ Error during preprocessing: {e}")
            return None

    def prepare_features_targets(self):
        """Prepare feature matrices and target variables"""
        if self.processed_data is None:
            print("❌ No processed data available")
            return None, None, None

        df = self.processed_data.copy()

        # Define feature columns
        feature_cols = [
            'location_encoded', 'year', 'month', 'day_of_year', 'week_of_year',
            'total_cases', 'new_cases', 'total_deaths', 'new_deaths',
            'population', 'population_density', 'median_age', 'aged_65_older',
            'gdp_per_capita', 'hospital_beds_per_thousand', 'stringency_index'
        ]

        # Add engineered features if they exist
        engineered_features = [
            'new_cases_7day_avg', 'new_cases_14day_avg', 'new_deaths_7day_avg',
            'total_cases_growth_rate', 'case_fatality_rate', 'tests_per_capita',
            'vaccination_rate'
        ]

        for feat in engineered_features:
            if feat in df.columns:
                feature_cols.append(feat)

        # Filter features that exist in the data
        available_features = [col for col in feature_cols if col in df.columns]

        X = df[available_features].copy()

        # Target variables
        targets = {
            'icu_demand': df['icu_demand'],
            'hospital_demand': df['hospital_demand'],
            'ventilator_demand': df['ventilator_demand'],
            'ppe_demand': df['ppe_demand']
        }

        print(f"📊 Features prepared: {len(available_features)} features")
        print(f"🎯 Targets prepared: {len(targets)} target variables")

        return X, targets, available_features

    def train_random_forest(self, X_train, y_train, target_name):
        """Train Random Forest model"""
        print(f"🌲 Training Random Forest for {target_name}...")

        rf = RandomForestRegressor(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            n_jobs=-1
        )

        rf.fit(X_train, y_train)
        self.models[f'rf_{target_name}'] = rf

        # Feature importance
        self.feature_importance[f'rf_{target_name}'] = dict(
            zip(X_train.columns, rf.feature_importances_)
        )

        return rf

    def train_xgboost(self, X_train, y_train, target_name):
        """Train XGBoost model"""
        print(f"🚀 Training XGBoost for {target_name}...")

        xgb_model = xgb.XGBRegressor(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            random_state=42,
            n_jobs=-1,
            verbosity=0
        )

        xgb_model.fit(X_train, y_train)
        self.models[f'xgb_{target_name}'] = xgb_model

        # Feature importance
        self.feature_importance[f'xgb_{target_name}'] = dict(
            zip(X_train.columns, xgb_model.feature_importances_)
        )

        return xgb_model

    def train_gradient_boosting(self, X_train, y_train, target_name):
        """Train Gradient Boosting model"""
        print(f"📈 Training Gradient Boosting for {target_name}...")

        gb = GradientBoostingRegressor(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            random_state=42
        )

        gb.fit(X_train, y_train)
        self.models[f'gb_{target_name}'] = gb

        # Feature importance
        self.feature_importance[f'gb_{target_name}'] = dict(
            zip(X_train.columns, gb.feature_importances_)
        )

        return gb

    def train_ensemble(self, X_train, y_train, target_name):
        """Train ensemble model combining RF, XGB, and GB"""
        print(f"🎭 Training Ensemble for {target_name}...")

        # Individual models
        rf = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
        xgb_model = xgb.XGBRegressor(n_estimators=50, random_state=42, verbosity=0)
        gb = GradientBoostingRegressor(n_estimators=50, random_state=42)

        # Ensemble
        ensemble = VotingRegressor([
            ('rf', rf),
            ('xgb', xgb_model),
            ('gb', gb)
        ])

        ensemble.fit(X_train, y_train)
        self.models[f'ensemble_{target_name}'] = ensemble

        return ensemble

    def evaluate_model(self, model, X_test, y_test):
        """Evaluate model performance"""
        try:
            y_pred = model.predict(X_test)

            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            return {
                'MSE': mse,
                'MAE': mae,
                'R2': r2,
                'RMSE': np.sqrt(mse)
            }
        except Exception as e:
            print(f"❌ Error evaluating model: {e}")
            return {'MSE': 0, 'MAE': 0, 'R2': 0, 'RMSE': 0}

    def train_all_models(self):
        """Train all models for all target variables"""
        print("🚀 Starting comprehensive model training...")

        X, targets, feature_names = self.prepare_features_targets()

        if X is None:
            print("❌ Cannot proceed without features")
            return {}

        # Scale features
        scaler = StandardScaler()
        X_scaled = pd.DataFrame(
            scaler.fit_transform(X),
            columns=X.columns,
            index=X.index
        )
        self.scalers['feature_scaler'] = scaler

        results = {}

        for target_name, y in targets.items():
            print(f"\n🎯 Training models for {target_name}...")

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(
                X_scaled, y, test_size=0.2, random_state=42
            )

            target_results = {}

            try:
                # Train Random Forest
                rf_model = self.train_random_forest(X_train, y_train, target_name)
                target_results['Random Forest'] = self.evaluate_model(rf_model, X_test, y_test)

                # Train XGBoost
                xgb_model = self.train_xgboost(X_train, y_train, target_name)
                target_results['XGBoost'] = self.evaluate_model(xgb_model, X_test, y_test)

                # Train Gradient Boosting
                gb_model = self.train_gradient_boosting(X_train, y_train, target_name)
                target_results['Gradient Boosting'] = self.evaluate_model(gb_model, X_test, y_test)

                # Train Ensemble
                ensemble_model = self.train_ensemble(X_train, y_train, target_name)
                target_results['Ensemble'] = self.evaluate_model(ensemble_model, X_test, y_test)

                results[target_name] = target_results
                print(f"✅ Completed training for {target_name}")

            except Exception as e:
                print(f"❌ Error training models for {target_name}: {e}")
                results[target_name] = {}

        self.training_results = results
        print("🎉 Model training completed!")
        return results

# PART 3: VISUALIZATION AND ANALYSIS FUNCTIONS
# ============================================================================

def analyze_feature_importance(predictor):
    """Analyze and visualize feature importance across all models"""

    print(f"\n{'='*60}")
    print("📊 FEATURE IMPORTANCE ANALYSIS")
    print(f"{'='*60}")

    if not hasattr(predictor, 'feature_importance') or not predictor.feature_importance:
        print("❌ Feature importance data not available. Please train models first.")
        return

    # Aggregate feature importance across all models
    all_features = {}

    for model_target, importance_dict in predictor.feature_importance.items():
        for feature, importance in importance_dict.items():
            if feature not in all_features:
                all_features[feature] = []
            all_features[feature].append(importance)

    # Calculate average importance
    avg_importance = {
        feature: np.mean(importances)
        for feature, importances in all_features.items()
    }

    # Sort by importance
    sorted_features = sorted(avg_importance.items(), key=lambda x: x[1], reverse=True)

    print("\n🏆 TOP 15 MOST IMPORTANT FEATURES (Average across all models):")
    print("-" * 70)
    print(f"{'Rank':<5} {'Feature':<30} {'Importance':<12} {'Bar'}")
    print("-" * 70)

    max_importance = sorted_features[0][1] if sorted_features else 1

    for i, (feature, importance) in enumerate(sorted_features[:15], 1):
        bar_length = int((importance / max_importance) * 20)
        bar = "█" * bar_length
        print(f"{i:<5} {feature:<30} {importance:<12.4f} {bar}")

def generate_predictions_report(predictor, country='United States', days_ahead=30):
    """Generate detailed predictions report for a specific country"""

    print(f"\n{'='*60}")
    print(f"📋 MEDICAL RESOURCE PREDICTION REPORT - {country}")
    print(f"{'='*60}")

    try:
        # Get country data
        if predictor.processed_data is None:
            print("❌ No processed data available")
            return

        country_data = predictor.processed_data[
            predictor.processed_data['location'] == country
        ].copy()

        if len(country_data) == 0:
            available_countries = predictor.processed_data['location'].unique()[:10]
            print(f"❌ No data available for {country}")
            print(f"📍 Available countries (sample): {list(available_countries)}")
            return

        latest_data = country_data.iloc[-1]

        print(f"📅 Report Date: {latest_data['date'].strftime('%Y-%m-%d')}")
        print(f"👥 Population: {latest_data['population']:,.0f}")

        # Current resource demands
        resources = {
            'ICU Beds': latest_data['icu_demand'],
            'Hospital Beds': latest_data['hospital_demand'],
            'Ventilators': latest_data['ventilator_demand'],
            'PPE Units': latest_data['ppe_demand']
        }

        print(f"\n📊 CURRENT RESOURCE DEMANDS:")
        print("-" * 40)
        for resource, demand in resources.items():
            print(f"{resource:<15}: {demand:>10,.0f}")

        # Simple predictions
        print(f"\n🔮 PREDICTED DEMANDS (+{days_ahead} days):")
        print("-" * 40)

        growth_rates = {'ICU Beds': 1.15, 'Hospital Beds': 1.10, 'Ventilators': 1.12, 'PPE Units': 1.20}

        for resource, current_demand in resources.items():
            predicted = current_demand * growth_rates[resource]
            change = ((predicted - current_demand) / current_demand) * 100 if current_demand > 0 else 0
            print(f"{resource:<15}: {predicted:>10,.0f} ({change:+.1f}%)")

    except Exception as e:
        print(f"❌ Error generating report: {str(e)}")

def create_visualizations(predictor):
    """Create comprehensive visualizations"""

    print(f"\n📊 Creating visualizations...")

    if predictor.processed_data is None:
        print("❌ No data available for visualization")
        return

    try:
        df = predictor.processed_data

        # Create subplots
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Medical Resource Demand Analysis', fontsize=16, fontweight='bold')

        # 1. Resource demand trends over time
        ax1 = axes[0, 0]
        resources_to_plot = ['icu_demand', 'hospital_demand', 'ventilator_demand']
        for resource in resources_to_plot:
            if resource in df.columns:
                monthly_avg = df.groupby(df['date'].dt.to_period('M'))[resource].mean()
                ax1.plot(range(len(monthly_avg)), monthly_avg.values,
                        label=resource.replace('_', ' ').title(), linewidth=2)
        ax1.set_title('Resource Demand Trends Over Time')
        ax1.set_xlabel('Time Period')
        ax1.set_ylabel('Average Demand')
        ax1.legend()

        # 2. ICU demand distribution
        ax2 = axes[0, 1]
        if 'icu_demand' in df.columns:
            ax2.hist(df['icu_demand'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
            ax2.set_title('ICU Demand Distribution')
            ax2.set_xlabel('ICU Demand')
            ax2.set_ylabel('Frequency')

        # 3. Top countries by ICU demand
        ax3 = axes[1, 0]
        if 'location' in df.columns and 'icu_demand' in df.columns:
            top_countries = df.groupby('location')['icu_demand'].max().nlargest(10)
            ax3.barh(range(len(top_countries)), top_countries.values, color='lightcoral')
            ax3.set_yticks(range(len(top_countries)))
            ax3.set_yticklabels([str(country)[:15] + '...' if len(str(country)) > 15 else str(country)
                                for country in top_countries.index])
            ax3.set_title('Top 10 Countries by Peak ICU Demand')
            ax3.set_xlabel('Peak ICU Demand')

        # 4. Model performance comparison
        ax4 = axes[1, 1]
        if hasattr(predictor, 'training_results') and predictor.training_results:
            models = []
            r2_scores = []

            for target, model_results in predictor.training_results.items():
                if target == 'icu_demand':
                    for model_name, metrics in model_results.items():
                        models.append(model_name[:10])  # Truncate long names
                        r2_scores.append(metrics.get('R2', 0))

            if models:
                colors = ['#FF9999', '#66B2FF', '#99FF99', '#FFCC99']
                bars = ax4.bar(models, r2_scores, color=colors[:len(models)])
                ax4.set_title('Model Performance (R² Score)')
                ax4.set_ylabel('R² Score')
                ax4.tick_params(axis='x', rotation=45)

                # Add value labels on bars
                for bar, score in zip(bars, r2_scores):
                    height = bar.get_height()
                    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                            f'{score:.3f}', ha='center', va='bottom')

        plt.tight_layout()
        plt.savefig('medical_resource_analysis.png', dpi=150, bbox_inches='tight')
        plt.show()

        print("✅ Visualizations created and saved as 'medical_resource_analysis.png'")

    except Exception as e:
        print(f"❌ Error creating visualizations: {e}")

# PART 4: MAIN EXECUTION FUNCTION
# ============================================================================

def main():
    """Main execution function"""
    print("🏥 MEDICAL RESOURCE DEMAND PREDICTION SYSTEM")
    print("=" * 60)
    print("🔧 Setting up environment...")

    # Setup Google Colab
    data_path = setup_google_colab()

    # Initialize predictor
    predictor = MedicalResourcePredictor(data_path)

    # Load and preprocess data
    print("\n📊 Loading and preprocessing data...")
    if predictor.load_data() is None:
        print("❌ Failed to load data. Exiting.")
        return None

    if predictor.preprocess_data() is None:
        print("❌ Failed to preprocess data. Exiting.")
        return None

    # Train all models
    print("\n🚀 Training machine learning models...")
    results = predictor.train_all_models()

    # Display results
    if results:
        print("\n" + "="*60)
        print("📊 TRAINING RESULTS SUMMARY")
        print("="*60)

        for target, models in results.items():
            print(f"\n🎯 {target.upper().replace('_', ' ')}:")
            for model_name, metrics in models.items():
                print(f"  📈 {model_name}:")
                print(f"    MAE: {metrics['MAE']:.2f}")
                print(f"    RMSE: {metrics['RMSE']:.2f}")
                print(f"    R²: {metrics['R2']:.3f}")

    # Generate additional analyses
    print("\n🔍 Generating additional analyses...")

    # Feature importance analysis
    analyze_feature_importance(predictor)

    # Generate prediction report for a sample country
    sample_countries = ['United States', 'United Kingdom', 'Germany', 'France', 'Italy']
    available_countries = predictor.processed_data['location'].unique()

    for country in sample_countries:
        if country in available_countries:
            generate_predictions_report(predictor, country=country)
            break
    else:
        print("\n⚠️  Sample countries not found in dataset. Using first available country.")
        generate_predictions_report(predictor, country=available_countries[0])

    # Create visualizations
    create_visualizations(predictor)

    print("\n🎉 Medical Resource Demand Prediction System completed successfully!")
    return predictor

# Execute the main function
if __name__ == "__main__":
    predictor = main()

# Addressing Data Quality

In [ ]:
# Handle missing data - we'll focus on countries with substantial data
country_coverage = df.groupby('location').apply(lambda x: x.isnull().mean())
good_coverage_countries = country_coverage[country_coverage['icu_patients_per_million'] < 0.7].index
df = df[df['location'].isin(good_coverage_countries)]

# Focus on key resource demand indicators
resource_metrics = [
    'icu_patients_per_million',
    'hosp_patients_per_million',
    'weekly_icu_admissions_per_million',
    'weekly_hosp_admissions_per_million'
]

# Plot resource metric trends
plt.figure(figsize=(12, 8))
for metric in resource_metrics:
    if metric in df.columns:
        sns.lineplot(data=df, x='date', y=metric, label=metric)
plt.title('Medical Resource Demand Metrics Over Time')
plt.ylabel('Per Million People')
plt.legend()
plt.show()

#Data Preparation
#Feature Engineering

In [ ]:
# Create temporal features
df['day_of_week'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['day_of_year'] = df['date'].dt.dayofyear

# Calculate rolling averages for key metrics
window_size = 7
for metric in resource_metrics:
    if metric in df.columns:
        df[f'{metric}_rolling_avg'] = df.groupby('location')[metric].transform(
            lambda x: x.rolling(window_size, min_periods=1).mean()
        )

# Create target variable - ICU demand 2 weeks ahead
df['future_icu_demand'] = df.groupby('location')['icu_patients_per_million'].shift(-14)

# Select relevant features
features = [
    'new_cases_per_million', 'new_deaths_per_million',
    'positive_rate', 'tests_per_case',
    'people_vaccinated_per_hundred', 'population_density',
    'median_age', 'aged_65_older', 'gdp_per_capita',
    'hospital_beds_per_thousand', 'day_of_week', 'month',
    'icu_patients_per_million_rolling_avg',
    'hosp_patients_per_million_rolling_avg'
]

# Filter dataframe
model_df = df[['location', 'date', 'future_icu_demand'] + features].dropna()

# Encode categorical variables
model_df = pd.get_dummies(model_df, columns=['location'], drop_first=True)

# Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X = model_df.drop(['future_icu_demand', 'date'], axis=1)
y = model_df['future_icu_demand']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=False)

# Implementation of Top Models

In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

# XGBoost Model
xgb_model = XGBRegressor(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb_model.fit(X_train, y_train)

# Random Forest Model
rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)

# Evaluation function
def evaluate_model(model, X_test, y_test):
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    rmse = sqrt(mean_squared_error(y_test, preds))
    print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}")
    return preds

print("XGBoost Performance:")
xgb_preds = evaluate_model(xgb_model, X_test, y_test)

print("\nRandom Forest Performance:")
rf_preds = evaluate_model(rf_model, X_test, y_test)

# Feature Importance Analysis

In [ ]:
# XGBoost Feature Importance
xgb_importance = xgb_model.feature_importances_
sorted_idx = np.argsort(xgb_importance)[-10:]  # Top 10 features

plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_idx)), xgb_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(X.columns)[sorted_idx])
plt.title('XGBoost Feature Importance')
plt.show()

#  Evaluation
# Model Comparison

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score

# Time-series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Cross-validation for XGBoost
xgb_scores = cross_val_score(
    xgb_model, X, y, cv=tscv, scoring='neg_mean_absolute_error')
print(f"XGBoost CV MAE: {-xgb_scores.mean():.2f} (± {xgb_scores.std():.2f})")

# Cross-validation for Random Forest
rf_scores = cross_val_score(
    rf_model, X, y, cv=tscv, scoring='neg_mean_absolute_error')
print(f"Random Forest CV MAE: {-rf_scores.mean():.2f} (± {rf_scores.std():.2f})")

In [ ]:
# Example of saving model for deployment
import joblib

joblib.dump(xgb_model, 'icu_demand_predictor.pkl')

# Example inference function
def predict_icu_demand(country, current_metrics):
    """Predict ICU demand 2 weeks ahead for a country"""
    # Preprocess input metrics
    input_df = preprocess_input(current_metrics)

    # Load model
    model = joblib.load('icu_demand_predictor.pkl')

    # Make prediction
    prediction = model.predict(input_df)

    return prediction

# Exploratory Data Analysis (EDA)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

# Mount Google Drive and load data
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/Medical_Resource_Prediction/owid-covid-data.csv'
df = pd.read_csv(data_path, parse_dates=['date'])

# Initial data inspection
print(f"Dataset dimensions: {df.shape}")
print("\nData types:\n", df.dtypes.value_counts())
print("\nMissing values percentage:\n", df.isnull().mean().sort_values(ascending=False).head(15))

# Temporal coverage analysis
print(f"\nDate range: {df['date'].min()} to {df['date'].max()}")

# Country-level data completeness
country_coverage = df.groupby('location').apply(lambda x: x[['hosp_patients_per_million',
                                                           'icu_patients_per_million']].notnull().mean())
plt.figure(figsize=(12, 6))
sns.heatmap(country_coverage.T, cmap='viridis')
plt.title('Hospitalization Data Coverage by Country')
plt.show()

# Resource metric distributions
resource_cols = ['hosp_patients_per_million', 'icu_patients_per_million',
                'weekly_hosp_admissions_per_million', 'weekly_icu_admissions_per_million']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
for col, ax in zip(resource_cols, axes.flatten()):
    if col in df.columns:
        sns.histplot(df[col].dropna(), kde=True, ax=ax)
        ax.set_title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

# Temporal trends for key metrics
plt.figure(figsize=(14, 8))
for metric in resource_cols:
    if metric in df.columns:
        sns.lineplot(data=df.groupby('date')[metric].mean().reset_index(),
                    x='date', y=metric, label=metric)
plt.title('Global Average Medical Resource Demand Over Time')
plt.ylabel('Patients per Million')
plt.legend()
plt.show()

# Correlation analysis
corr_matrix = df[resource_cols + ['new_cases_smoothed_per_million',
                                'people_fully_vaccinated_per_hundred',
                                'stringency_index',
                                'hospital_beds_per_thousand']].corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Key Variables')
plt.show()

# Data Preparation

In [ ]:
# Feature engineering
def prepare_features(df):
    # Create temporal features
    df['day_of_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['day_of_year'] = df['date'].dt.dayofyear

    # Calculate growth rates
    df['case_growth_7day'] = df.groupby('location')['new_cases_smoothed_per_million'].pct_change(7)
    df['hosp_growth_7day'] = df.groupby('location')['hosp_patients_per_million'].pct_change(7)

    # Vaccination effectiveness metric
    df['vax_effectiveness'] = (df['people_fully_vaccinated_per_hundred'] * 0.7 +
                              df['total_boosters_per_hundred'] * 0.3)

    # Healthcare strain index
    df['strain_index'] = (df['hosp_patients_per_million'] /
                         df['hospital_beds_per_thousand']) * 1000

    # Wave detection
    df['pandemic_wave'] = (df['new_cases_smoothed_per_million'].rolling(14).mean() > 100).astype(int)

    # Target variables - 14-day ahead forecast
    df['future_hosp_demand'] = df.groupby('location')['hosp_patients_per_million'].shift(-14)
    df['future_icu_demand'] = df.groupby('location')['icu_patients_per_million'].shift(-14)

    return df

df = prepare_features(df)

# Select complete cases
model_df = df[['location', 'date', 'future_hosp_demand', 'future_icu_demand',
              'new_cases_smoothed_per_million', 'case_growth_7day',
              'positive_rate', 'reproduction_rate',
              'vax_effectiveness', 'strain_index',
              'hospital_beds_per_thousand', 'median_age',
              'aged_65_older', 'stringency_index',
              'pandemic_wave']].dropna()

# Encode categorical variables
model_df = pd.get_dummies(model_df, columns=['location'], drop_first=True)

# Temporal train-test split
train = model_df[model_df['date'] < '2022-01-01']
test = model_df[model_df['date'] >= '2022-01-01']

X_train = train.drop(['date', 'future_hosp_demand', 'future_icu_demand'], axis=1)
y_train_hosp = train['future_hosp_demand']
y_train_icu = train['future_icu_demand']

X_test = test.drop(['date', 'future_hosp_demand', 'future_icu_demand'], axis=1)
y_test_hosp = test['future_hosp_demand']
y_test_icu = test['future_icu_demand']

In [ ]:
# List columns related to healthcare resources and potential predictors
healthcare_cols = [col for col in df.columns if any(term in col.lower() for term in
                   ['hospital', 'icu', 'ventilator', 'bed', 'patient', 'medical',
                    'healthcare', 'resource', 'admission'])]

case_cols = [col for col in df.columns if any(term in col.lower() for term in
             ['case', 'death', 'positive', 'test', 'confirmed'])]

demographic_cols = [col for col in df.columns if any(term in col.lower() for term in
                   ['population', 'density', 'age', 'gdp', 'income', 'poverty'])]

# Display identified columns
print("Healthcare resource columns:", healthcare_cols)
print("Case/infection columns:", case_cols)
print("Demographic columns:", demographic_cols)

In [ ]:
# Select a few representative countries
countries = ['United States', 'United Kingdom', 'India', 'Brazil', 'South Africa']

# Plot hospital admissions or ICU patients over time (if available)
plt.figure(figsize=(14, 8))
for country in countries:
    country_data = df[df['location'] == country]
    if 'hosp_patients' in df.columns:
        plt.plot(pd.to_datetime(country_data['date']),
                 country_data['hosp_patients'], label=country)

plt.title('COVID-19 Hospital Patients Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Patients')
plt.legend()
plt.grid(True)
plt.show()

# Plot new cases vs. hospital admissions (if available)
if 'new_cases' in df.columns and 'weekly_hosp_admissions' in df.columns:
    plt.figure(figsize=(14, 8))
    for country in countries:
        country_data = df[df['location'] == country].dropna(subset=['new_cases', 'weekly_hosp_admissions'])
        plt.scatter(country_data['new_cases'], country_data['weekly_hosp_admissions'],
                    alpha=0.6, label=country)

    plt.title('New Cases vs. Hospital Admissions')
    plt.xlabel('New Cases')
    plt.ylabel('Weekly Hospital Admissions')
    plt.legend()
    plt.grid(True)
    plt.show()

# Correlation Analysis

In [ ]:
# Select relevant numeric columns
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
relevant_cols = [col for col in numeric_cols if col not in ['iso_code']]

# Calculate correlation matrix
corr_matrix = df[relevant_cols].corr()

# Visualize correlations with hospitalization data
plt.figure(figsize=(16, 12))
hosp_cols = [col for col in relevant_cols if 'hosp' in col or 'icu' in col]
if hosp_cols:
    hosp_corr = corr_matrix[hosp_cols].sort_values(by=hosp_cols[0], ascending=False)
    sns.heatmap(hosp_corr.head(15), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Correlation with Hospitalization Data')
    plt.tight_layout()
    plt.show()

# Data cleaning

In [ ]:
# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Identify key target variables for prediction
target_variables = [col for col in df.columns if col in
                   ['hosp_patients', 'icu_patients', 'weekly_hosp_admissions',
                    'weekly_icu_admissions', 'hospital_beds_per_thousand']]

# Handle missing values in target variables
# For time series data, forward-fill might be appropriate
for col in target_variables:
    if col in df.columns:
        # Group by location and forward fill
        df[col] = df.groupby('location')[col].transform(
            lambda x: x.ffill().bfill()
        )

# Filter countries with sufficient healthcare data
if target_variables:
    min_data_points = 30  # Minimum required data points
    valid_countries = df.groupby('location').apply(
        lambda x: x[target_variables].notna().sum().min() >= min_data_points
    )
    valid_countries = valid_countries[valid_countries].index.tolist()

    print(f"Number of countries with sufficient healthcare data: {len(valid_countries)}")
    df_filtered = df[df['location'].isin(valid_countries)]
else:
    print("No target healthcare variables found, using complete dataset")
    df_filtered = df.copy()

# Handle outliers in key variables
def handle_outliers(df, column, lower_quantile=0.001, upper_quantile=0.999):
    if column in df.columns:
        q_low = df[column].quantile(lower_quantile)
        q_high = df[column].quantile(upper_quantile)
        df[column] = df[column].clip(lower=q_low, upper=q_high)
    return df

# Apply outlier handling to case, death and hospitalization data
for col in ['new_cases', 'new_deaths'] + target_variables:
    if col in df.columns:
        df_filtered = handle_outliers(df_filtered, col)

# Remove unnecessary columns
cols_to_drop = ['iso_code', 'tests_units']  # Add more as needed
df_filtered = df_filtered.drop(columns=[col for col in cols_to_drop if col in df_filtered.columns])

# Feature Engineering
# Creating new features that might help predict resource demand

In [ ]:
# Add time-based features
df_filtered['year'] = df_filtered['date'].dt.year
df_filtered['month'] = df_filtered['date'].dt.month
df_filtered['week'] = df_filtered['date'].dt.isocalendar().week
df_filtered['day_of_week'] = df_filtered['date'].dt.dayofweek

# Calculate rolling metrics for cases and deaths
for window in [7, 14, 30]:
    # Group by location and calculate rolling averages
    df_filtered[f'new_cases_rolling_{window}d'] = df_filtered.groupby('location')['new_cases'].transform(
        lambda x: x.rolling(window=window, min_periods=1).mean()
    )

    df_filtered[f'new_deaths_rolling_{window}d'] = df_filtered.groupby('location')['new_deaths'].transform(
        lambda x: x.rolling(window=window, min_periods=1).mean()
    )

# Calculate rate of change (acceleration/deceleration)
df_filtered['case_growth_rate'] = df_filtered.groupby('location')['new_cases_rolling_7d'].transform(
    lambda x: x.pct_change()
)

# Calculate case fatality rate (CFR)
df_filtered['case_fatality_rate'] = df_filtered['total_deaths'] / df_filtered['total_cases']

# Create lagged features (cases tend to precede hospitalizations)
for lag in [7, 14, 21]:
    df_filtered[f'new_cases_lag_{lag}d'] = df_filtered.groupby('location')['new_cases'].transform(
        lambda x: x.shift(lag)
    )

    df_filtered[f'new_deaths_lag_{lag}d'] = df_filtered.groupby('location')['new_deaths'].transform(
        lambda x: x.shift(lag)
    )

# Create features representing healthcare capacity
if 'hospital_beds_per_thousand' in df_filtered.columns and 'population' in df_filtered.columns:
    # Estimate total bed capacity
    df_filtered['estimated_total_beds'] = (
        df_filtered['hospital_beds_per_thousand'] * df_filtered['population'] / 1000
    )

# Population demographic features
age_cols = [col for col in df_filtered.columns if 'aged' in col]
if age_cols:
    # Create composite risk score based on age demographics
    # Higher weights for older population groups which typically require more resources
    weights = {
        'aged_65_older': 1.0,
        'aged_70_older': 1.5,
        'population_80plus': 2.0  # If this column exists
    }

    # Calculate age risk score
    for col, weight in weights.items():
        if col in df_filtered.columns:
            if 'age_risk_score' not in df_filtered.columns:
                df_filtered['age_risk_score'] = 0
            df_filtered['age_risk_score'] += df_filtered[col] * weight

# Feature Selection
# Selecting the most relevant features for modeling:

In [ ]:
# Identify potential predictor variables
predictors = [
    # Case and death metrics
    'new_cases', 'new_deaths', 'total_cases', 'total_deaths',
    'new_cases_per_million', 'new_deaths_per_million',
    'case_fatality_rate',

    # Rolling averages and lagged features
    'new_cases_rolling_7d', 'new_cases_rolling_14d', 'new_cases_rolling_30d',
    'new_deaths_rolling_7d', 'new_deaths_rolling_14d', 'new_deaths_rolling_30d',
    'new_cases_lag_7d', 'new_cases_lag_14d', 'new_cases_lag_21d',
    'case_growth_rate',

    # Healthcare capacity
    'hospital_beds_per_thousand', 'estimated_total_beds',

    # Demographics
    'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
    'age_risk_score',

    # GDP and healthcare indicators
    'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate', 'diabetes_prevalence',
    'female_smokers', 'male_smokers', 'handwashing_facilities',

    # Time-based features
    'month', 'week', 'day_of_week'
]

# Filter to include only columns that actually exist in the dataset
predictors = [col for col in predictors if col in df_filtered.columns]

# Check feature correlations
correlation_matrix = df_filtered[predictors].corr()

# Identify highly correlated features (absolute correlation > 0.9)
high_corr_pairs = []
for i in range(len(predictors)):
    for j in range(i+1, len(predictors)):
        if abs(correlation_matrix.iloc[i, j]) > 0.9:
            high_corr_pairs.append((predictors[i], predictors[j], correlation_matrix.iloc[i, j]))

print("Highly correlated feature pairs:")
for pair in high_corr_pairs:
    print(f"{pair[0]} and {pair[1]}: {pair[2]:.2f}")

# Function to select target variable based on available data
def select_target_variable(df, possible_targets):
    for target in possible_targets:
        if target in df.columns and df[target].notna().sum() > len(df) * 0.5:
            return target
    return None

# Select target variable based on availability
target_priorities = [
    'hosp_patients',              # Current hospitalized patients
    'weekly_hosp_admissions',     # Weekly new hospital admissions
    'icu_patients',               # Current ICU patients
    'weekly_icu_admissions'       # Weekly new ICU admissions
]

target_variable = select_target_variable(df_filtered, target_priorities)
print(f"Selected target variable: {target_variable}")

# If no healthcare target variable is available, we cannot proceed with this specific prediction task
if target_variable is None:
    print("No suitable target variable found in the dataset.")
    # In a real scenario, we might need to obtain additional data or redefine the prediction task

# Data Splitting and Transformation
# Preparing the data for modeling by splitting into train/validation/test sets and applying necessary transformations:

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
import numpy as np
import pandas as pd

# Prepare feature matrix and target vector
X = df_filtered[predictors].copy()
if target_variable:
    y = df_filtered[target_variable].copy()

    # Fill remaining NaN values in features with appropriate methods
    for col in X.columns:
        # For rate columns, fill NaN with 0
        if 'rate' in col or 'pct' in col:
            X[col] = X[col].fillna(0)
        # For other columns, fill with median
        else:
            X[col] = X[col].fillna(X[col].median())

    # Replace infinite values with NaN, then handle them
    X.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Handle any remaining NaN values after replacing infinities
    for col in X.columns:
        if 'rate' in col or 'pct' in col:
            X[col] = X[col].fillna(0)
        else:
            X[col] = X[col].fillna(X[col].median())

    # Find rows where target is not NaN
    valid_indices = y.notna()
    X = X[valid_indices]
    y = y[valid_indices]

    # Handle any remaining NaN values in the target
    y = y.fillna(y.median())

    # Create country indicators for modeling
    country_dummies = pd.get_dummies(df_filtered.loc[valid_indices, 'location'], prefix='country')

    # Add country indicators to features (optional, depending on model approach)
    # X = pd.concat([X, country_dummies], axis=1)

    # Split the data into training and temporary set (80% train, 20% temp)
    # For time series data, a chronological split is often better
    # Here we split by time for each country

    # First, create a unique identifier for each country-date combination
    df_filtered['country_date'] = df_filtered['location'] + '_' + df_filtered['date'].astype(str)

    # Get unique countries
    countries = df_filtered['location'].unique()

    # Initialize empty DataFrames for train, validation, and test sets
    X_train = pd.DataFrame()
    X_val = pd.DataFrame()
    X_test = pd.DataFrame()
    y_train = pd.Series()
    y_val = pd.Series()
    y_test = pd.Series()

    # Split the data for each country
    for country in countries:
        country_mask = df_filtered['location'] == country
        country_data = df_filtered[country_mask].sort_values('date')

        if country_data[target_variable].notna().sum() > 30:  # Only use countries with sufficient data
            # Extract features and target
            country_X = country_data[predictors].copy()
            country_y = country_data[target_variable].copy()

            # Handle NaN values
            for col in country_X.columns:
                if 'rate' in col or 'pct' in col:
                    country_X[col] = country_X[col].fillna(0)
                else:
                    country_X[col] = country_X[col].fillna(country_X[col].median())

            # Replace infinite values with NaN, then fill
            country_X.replace([np.inf, -np.inf], np.nan, inplace=True)

            # Handle any remaining NaN values after replacing infinities
            for col in country_X.columns:
                if 'rate' in col or 'pct' in col:
                    country_X[col] = country_X[col].fillna(0)
                else:
                    country_X[col] = country_X[col].fillna(country_X[col].median())

            country_y = country_y.fillna(country_y.median())

            # Find valid rows
            valid_indices = country_y.notna()
            country_X = country_X[valid_indices]
            country_y = country_y[valid_indices]

            # Calculate split indices
            n = len(country_X)
            train_idx = int(0.7 * n)
            val_idx = int(0.85 * n)

            # Split the data
            X_train = pd.concat([X_train, country_X.iloc[:train_idx]])
            X_val = pd.concat([X_val, country_X.iloc[train_idx:val_idx]])
            X_test = pd.concat([X_test, country_X.iloc[val_idx:]])

            y_train = pd.concat([y_train, country_y.iloc[:train_idx]])
            y_val = pd.concat([y_val, country_y.iloc[train_idx:val_idx]])
            y_test = pd.concat([y_test, country_y.iloc[val_idx:]])

    # Add a verification step to check for infinities or extremely large values before scaling
    def check_and_clean_dataframe(df):
        """Check for and handle problematic values in the dataframe"""
        # Replace infinities with NaN
        df.replace([np.inf, -np.inf], np.nan, inplace=True)

        # Check for and cap extremely large values (using a reasonable threshold)
        for col in df.columns:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            upper_bound = q3 + 10 * iqr  # Less strict than usual 1.5*IQR to keep more data
            lower_bound = q1 - 10 * iqr

            # Cap extreme values
            df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)

            # Fill any remaining NaNs
            if 'rate' in col or 'pct' in col:
                df[col] = df[col].fillna(0)
            else:
                df[col] = df[col].fillna(df[col].median())

        return df

    # Apply cleaning to all datasets
    X_train = check_and_clean_dataframe(X_train)
    X_val = check_and_clean_dataframe(X_val)
    X_test = check_and_clean_dataframe(X_test)

    # Scale the features
    scaler = RobustScaler()  # RobustScaler is less sensitive to outliers
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # Convert back to DataFrame for better handling
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns, index=X_val.index)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

    print(f"Training set: {X_train_scaled.shape[0]} samples")
    print(f"Validation set: {X_val_scaled.shape[0]} samples")
    print(f"Test set: {X_test_scaled.shape[0]} samples")

#Prophet Model

In [ ]:
from prophet import Prophet
import pandas as pd

def train_prophet_model(df, target_column, horizon=30):
    # Prepare data in Prophet format
    prophet_df = df[['date', target_column]].rename(
        columns={'date': 'ds', target_column: 'y'})

    # Initialize and train model with appropriate parameters
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=0.05,  # Flexibility in trend changes
        seasonality_prior_scale=10,    # Stronger seasonality component
        holidays_prior_scale=10        # Important for medical demand
    )

    # Add country-specific holidays if available
    if 'country' in df.columns:
        country = df['country'].iloc[0]
        # Add relevant holidays for this country
        # code for adding holidays...

    # Fit the model
    model.fit(prophet_df)

    # Create future dataframe for predictions
    future = model.make_future_dataframe(periods=horizon)

    # Generate forecast
    forecast = model.predict(future)

    return model, forecast

In [ ]:
def visualize_prophet_forecast(model, forecast):
    """Visualize Prophet forecast with components"""
    # Main forecast plot
    fig1 = model.plot(forecast)
    plt.title('Prophet Forecast for Medical Resource Demand')
    plt.xlabel('Date')
    plt.ylabel('Resource Demand')
    plt.show()

    # Trend and seasonality components
    fig2 = model.plot_components(forecast)
    plt.suptitle('Prophet Forecast Components', y=1.02)
    plt.show()

    # Interactive plot (if in notebook)
    try:
        from prophet.plot import plot_plotly, plot_components_plotly
        plot_plotly(model, forecast)
        plot_components_plotly(model, forecast)
    except:
        pass

# XGBoost

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

def train_xgboost_model(X_train, y_train, X_test, y_test):
    # Parameter grid optimized for medical resource prediction
    params = {
        'objective': 'reg:squarederror',
        'learning_rate': 0.05,
        'max_depth': 6,
        'min_child_weight': 2,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'gamma': 0.1,
        'reg_alpha': 0.1,
        'reg_lambda': 1.0,
        'n_estimators': 500
    }

    # Initialize model
    model = xgb.XGBRegressor(**params)

    # Time-series cross-validation to respect temporal order
    tscv = TimeSeriesSplit(n_splits=5)

    # Early stopping to prevent overfitting
    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_metric=['rmse', 'mae'],
        early_stopping_rounds=20,
        verbose=False
    )

    # Get feature importance
    feature_importance = model.feature_importances_
    feature_names = X_train.columns
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importance
    }).sort_values(by='Importance', ascending=False)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)

    return model, importance_df, rmse, mae

In [ ]:
def visualize_xgboost_results(y_test, y_pred, importance_df, model_name="XGBoost"):
    """Visualize XGBoost predictions and feature importance"""
    plt.figure(figsize=(15, 5))

    # Actual vs Predicted
    plt.subplot(1, 2, 1)
    plt.plot(y_test.values, label='Actual')
    plt.plot(y_pred, label='Predicted', alpha=0.7)
    plt.title(f'{model_name} - Actual vs Predicted')
    plt.xlabel('Time Steps')
    plt.ylabel('Resource Demand')
    plt.legend()

    # Feature Importance
    plt.subplot(1, 2, 2)
    sns.barplot(x='Importance', y='Feature',
               data=importance_df.head(15))
    plt.title(f'{model_name} - Top 15 Feature Importance')
    plt.tight_layout()
    plt.show()

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def train_random_forest(X_train, y_train, X_test, y_test):
    # Initialize model with parameters suited for medical data
    rf_model = RandomForestRegressor(
        n_estimators=200,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        max_features='sqrt',
        bootstrap=True,
        random_state=42,
        n_jobs=-1  # Use all available cores
    )

    # Train model
    rf_model.fit(X_train, y_train)

    # Evaluate
    y_pred = rf_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)

    # Extract feature importance
    feature_importance = rf_model.feature_importances_
    feature_names = X_train.columns
    rf_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importance
    }).sort_values(by='Importance', ascending=False)

    return rf_model, rf_importance, rmse, mae

In [ ]:
def visualize_random_forest(y_test, y_pred, importance_df):
    """Visualize Random Forest results"""
    plt.figure(figsize=(15, 5))

    # Actual vs Predicted
    plt.subplot(1, 2, 1)
    plt.scatter(y_test, y_pred, alpha=0.3)
    plt.plot([y_test.min(), y_test.max()],
             [y_test.min(), y_test.max()], 'r--')
    plt.title('Random Forest - Actual vs Predicted')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')

    # Feature Importance
    plt.subplot(1, 2, 2)
    sns.barplot(x='Importance', y='Feature',
               data=importance_df.head(15))
    plt.title('Random Forest - Top 15 Feature Importance')
    plt.tight_layout()
    plt.show()

# LSTM (Long Short-Term Memory Networks)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

def prepare_lstm_data(data, target_col, lookback=14):
    """Transform data into sequences suitable for LSTM"""
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:(i + lookback)])
        y.append(data[i + lookback, target_col])
    return np.array(X), np.array(y)

def build_lstm_model(X_train):
    """Build and compile an LSTM model for medical resource prediction"""
    # Input shape: [samples, time steps, features]
    input_shape = (X_train.shape[1], X_train.shape[2])

    model = Sequential([
        # First LSTM layer with return sequences for stacking
        LSTM(64, activation='relu', return_sequences=True,
             input_shape=input_shape),
        Dropout(0.2),  # Prevent overfitting

        # Second LSTM layer
        LSTM(32, activation='relu'),
        Dropout(0.2),

        # Output layer
        Dense(1)  # Single output for resource demand prediction
    ])

    # Compile model with appropriate loss function
    model.compile(
        optimizer='adam',
        loss='mean_squared_error',
        metrics=['mae']
    )

    return model

def train_lstm_model(X_train, y_train, X_val, y_val, epochs=100, batch_size=32):
    """Train LSTM model with early stopping"""
    # Build model
    model = build_lstm_model(X_train)

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1
    )

    return model, history

In [ ]:
def visualize_lstm_results(history, y_test, y_pred):
    """Visualize LSTM training history and predictions"""
    plt.figure(figsize=(15, 5))

    # Training history
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('LSTM Training History')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Predictions vs Actual
    plt.subplot(1, 2, 2)
    plt.plot(y_test, label='Actual')
    plt.plot(y_pred, label='Predicted', alpha=0.7)
    plt.title('LSTM - Actual vs Predicted')
    plt.xlabel('Time Steps')
    plt.ylabel('Resource Demand')
    plt.legend()
    plt.tight_layout()
    plt.show()

# Ensemble Approach for Improved Robustness

In [ ]:
def create_ensemble_prediction(prophet_pred, xgb_pred, rf_pred, lstm_pred, weights=None):
    """
    Combine predictions from multiple models with optional weighting
    """
    if weights is None:
        # Equal weighting
        weights = [0.25, 0.25, 0.25, 0.25]

    # Ensure weights sum to 1
    weights = np.array(weights) / sum(weights)

    # Combine predictions
    ensemble_pred = (
        weights[0] * prophet_pred +
        weights[1] * xgb_pred +
        weights[2] * rf_pred +
        weights[3] * lstm_pred
    )

    return ensemble_pred

In [ ]:
def visualize_ensemble_results(y_test, ensemble_pred, individual_preds):
    """Visualize ensemble vs individual model performance"""
    plt.figure(figsize=(12, 6))

    # Plot all predictions
    plt.plot(y_test.values, label='Actual', linewidth=2)
    plt.plot(ensemble_pred, label='Ensemble', linewidth=2)

    # Plot individual model predictions
    for model_name, pred in individual_preds.items():
        plt.plot(pred, label=model_name, alpha=0.5, linestyle='--')

    plt.title('Ensemble vs Individual Model Predictions')
    plt.xlabel('Time Steps')
    plt.ylabel('Resource Demand')
    plt.legend()
    plt.grid(True)
    plt.show()

# Model Evaluation and Selection
# To select the best model for our healthcare resource prediction task:

In [ ]:
def evaluate_models(models, X_test, y_test, target_cols):
    """Evaluate multiple models on test data"""
    results = {}

    for model_name, model in models.items():
        y_pred = model.predict(X_test)

        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)

        # Domain-specific metrics for healthcare
        # Underprediction is more costly than overprediction
        under_pred = np.mean((y_test - y_pred)[y_test > y_pred])
        over_pred = np.mean((y_pred - y_test)[y_pred > y_test])

        # Store results
        results[model_name] = {
            'RMSE': rmse,
            'MAE': mae,
            'Under-prediction Error': under_pred,
            'Over-prediction Error': over_pred
        }

    return pd.DataFrame(results).T

# Model Interpretability
# For healthcare applications, model interpretability is crucial:

In [ ]:
import shap

def interpret_xgboost_model(model, X_test):
    """Generate SHAP values for XGBoost model interpretation"""
    # Initialize SHAP explainer
    explainer = shap.TreeExplainer(model)

    # Calculate SHAP values
    shap_values = explainer.shap_values(X_test)

    # Create summary plot
    shap.summary_plot(shap_values, X_test, plot_type="bar")

    # Return values for detailed analysis
    return explainer, shap_values

# ERROR METRICS VISUALIZATION

In [ ]:
def visualize_error_metrics(metrics_df):
    """Visualize model comparison metrics"""
    plt.figure(figsize=(12, 6))

    # RMSE and MAE
    plt.subplot(1, 2, 1)
    metrics_df[['RMSE', 'MAE']].plot(kind='bar')
    plt.title('Model Error Comparison')
    plt.ylabel('Error Value')
    plt.xticks(rotation=45)

    # Under/Over prediction
    plt.subplot(1, 2, 2)
    metrics_df[['Under-prediction Error', 'Over-prediction Error']].plot(kind='bar')
    plt.title('Under/Over Prediction Comparison')
    plt.ylabel('Error Value')
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

# SHAP Value Visualization (XGBoost Interpretability)

In [ ]:
def visualize_shap_values(explainer, shap_values, X_test):
    """Visualize SHAP values for model interpretability"""
    plt.figure(figsize=(15, 10))

    # Summary plot
    plt.subplot(2, 2, 1)
    shap.summary_plot(shap_values, X_test, plot_type="bar")
    plt.title('Feature Importance (SHAP)')

    # Detailed SHAP values
    plt.subplot(2, 2, 2)
    shap.summary_plot(shap_values, X_test)
    plt.title('SHAP Value Distribution')

    # Dependence plot for top feature
    top_feature = X_test.columns[np.argmax(np.abs(shap_values).mean(0))]
    plt.subplot(2, 2, 3)
    shap.dependence_plot(top_feature, shap_values, X_test)
    plt.title(f'Dependence Plot for {top_feature}')

    # Force plot for single prediction
    plt.subplot(2, 2, 4)
    shap.force_plot(explainer.expected_value, shap_values[0,:], X_test.iloc[0,:])
    plt.title('Force Plot for Single Prediction')

    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Assuming you have your data loaded as df
# Let's create a sample target variable for demonstration
df['resource_demand'] = df['hosp_patients'].fillna(0)  # Using hospital patients as target

# Split data into train and test (time-series split)
train_size = int(len(df) * 0.8)
train = df.iloc[:train_size]
test = df.iloc[train_size:]

# 1. Train Prophet Model
def train_prophet_model(train_df, target_col='resource_demand'):
    prophet_df = train_df[['date', target_col]].rename(columns={'date': 'ds', target_col: 'y'})
    model = Prophet(yearly_seasonality=True, weekly_seasonality=True)
    model.fit(prophet_df)
    future = model.make_future_dataframe(periods=len(test))
    forecast = model.predict(future)
    return model, forecast

prophet_model, prophet_forecast = train_prophet_model(train)

# 2. Train XGBoost Model
def train_xgboost_model(train_df, test_df, features, target_col='resource_demand'):
    model = xgb.XGBRegressor()
    model.fit(train_df[features], train_df[target_col])
    preds = model.predict(test_df[features])
    return model, preds

# Select features - you should customize this based on your actual features
features = ['total_cases', 'new_cases', 'population']
xgb_model, xgb_preds = train_xgboost_model(train, test, features)

# 3. Train Random Forest Model
def train_random_forest(train_df, test_df, features, target_col='resource_demand'):
    model = RandomForestRegressor()
    model.fit(train_df[features], train_df[target_col])
    preds = model.predict(test_df[features])
    return model, preds

rf_model, rf_preds = train_random_forest(train, test, features)

# Now you can visualize
def visualize_prophet_forecast(model, forecast, actual_data):
    fig = model.plot(forecast)
    plt.plot(actual_data['date'], actual_data['resource_demand'], 'r.', label='Actual')
    plt.title('Prophet Forecast vs Actual')
    plt.legend()
    plt.show()

def visualize_xgboost_results(y_true, y_pred):
    plt.figure(figsize=(10, 5))
    plt.plot(y_true, label='Actual')
    plt.plot(y_pred, label='XGBoost Predicted', alpha=0.7)
    plt.title('XGBoost Predictions')
    plt.legend()
    plt.show()

def visualize_rf_results(y_true, y_pred):
    plt.figure(figsize=(10, 5))
    plt.scatter(y_true, y_pred, alpha=0.3)
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--')
    plt.title('Random Forest Predictions')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.show()

# Now call the visualization functions with your trained models
visualize_prophet_forecast(prophet_model, prophet_forecast, test)
visualize_xgboost_results(test['resource_demand'], xgb_preds)
visualize_rf_results(test['resource_demand'], rf_preds)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Function to examine dataset before and after cleaning
def examine_dataset_cleaning(df_original, column_subset=None):
    """
    Examine a dataset before and after cleaning to identify problematic values.

    Args:
        df_original: The original dataframe to be examined
        column_subset: Optional list of specific columns to examine (None for all columns)

    Returns:
        df_cleaned: The cleaned dataframe
    """
    # Create a copy to avoid modifying the original
    df = df_original.copy()

    # Select columns to examine (all or subset)
    if column_subset is not None and all(col in df.columns for col in column_subset):
        examine_cols = column_subset
    else:
        examine_cols = df.columns

    # Check dimensions
    print(f"Original dataset shape: {df.shape}")

    # 1. Check for missing values
    missing_vals = df[examine_cols].isna().sum()
    print("\nMissing values before cleaning:")
    print(missing_vals[missing_vals > 0])

    # 2. Check for infinities
    inf_counts = {}
    for col in examine_cols:
        inf_count = np.isinf(df[col]).sum()
        if inf_count > 0:
            inf_counts[col] = inf_count

    if inf_counts:
        print("\nColumns with infinity values:")
        for col, count in inf_counts.items():
            print(f"{col}: {count} infinity values")
    else:
        print("\nNo infinity values found.")

    # 3. Identify extremely large values (potential outliers)
    print("\nExtreme values before cleaning:")
    for col in examine_cols:
        if df[col].dtype.kind in 'ifc':  # Only check numeric columns
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            upper_bound = q3 + 5 * iqr
            lower_bound = q1 - 5 * iqr

            extreme_high = df[df[col] > upper_bound][col]
            extreme_low = df[df[col] < lower_bound][col]

            if len(extreme_high) > 0 or len(extreme_low) > 0:
                print(f"\n{col}:")
                print(f"  Normal range: [{lower_bound:.2f}, {upper_bound:.2f}]")
                if len(extreme_high) > 0:
                    print(f"  High outliers: {len(extreme_high)} values")
                    print(f"  Max value: {extreme_high.max()}")
                if len(extreme_low) > 0:
                    print(f"  Low outliers: {len(extreme_low)} values")
                    print(f"  Min value: {extreme_low.min()}")

    # Clean the dataset
    print("\n--- Cleaning Dataset ---")
    df_cleaned = clean_dataset(df, examine_cols)

    # Check dimensions after cleaning
    print(f"\nCleaned dataset shape: {df_cleaned.shape}")

    # Check for missing values after cleaning
    missing_vals_after = df_cleaned[examine_cols].isna().sum()
    if missing_vals_after.sum() > 0:
        print("\nRemaining missing values after cleaning:")
        print(missing_vals_after[missing_vals_after > 0])
    else:
        print("\nNo missing values after cleaning.")

    # Check for infinities after cleaning
    has_inf_after = False
    for col in examine_cols:
        if np.isinf(df_cleaned[col]).sum() > 0:
            has_inf_after = True
            break

    if has_inf_after:
        print("\nWarning: Infinities still present after cleaning!")
    else:
        print("\nNo infinity values after cleaning.")

    # Visualize data distribution before and after cleaning for a sample of columns
    visualize_distributions(df, df_cleaned, examine_cols)

    return df_cleaned

def clean_dataset(df, columns):
    """
    Clean the dataset by handling missing values, infinities, and extreme values.
    """
    df_clean = df.copy()

    # Replace infinities with NaN
    df_clean.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Handle extreme values and remaining NaNs
    for col in columns:
        if df_clean[col].dtype.kind in 'ifc':  # Only process numeric columns
            # Calculate bounds
            q1 = df_clean[col].dropna().quantile(0.25)
            q3 = df_clean[col].dropna().quantile(0.75)
            iqr = q3 - q1
            upper_bound = q3 + 10 * iqr  # Using a generous threshold
            lower_bound = q1 - 10 * iqr

            # Cap extreme values
            df_clean[col] = df_clean[col].clip(lower=lower_bound, upper=upper_bound)

            # Fill remaining NaNs
            if 'rate' in col or 'pct' in col:
                df_clean[col] = df_clean[col].fillna(0)
            else:
                df_clean[col] = df_clean[col].fillna(df_clean[col].median())

    return df_clean

def visualize_distributions(df_original, df_cleaned, columns, max_cols=5):
    """
    Create histograms to compare distributions before and after cleaning.
    """
    # Select a subset of columns if there are too many
    if len(columns) > max_cols:
        # Prioritize columns with known issues for visualization
        cols_to_plot = []

        # First, add columns with infinities
        for col in columns:
            if np.isinf(df_original[col]).sum() > 0 and len(cols_to_plot) < max_cols:
                cols_to_plot.append(col)

        # Then add columns with extreme values
        for col in columns:
            if col not in cols_to_plot and df_original[col].dtype.kind in 'ifc':
                q1 = df_original[col].quantile(0.25)
                q3 = df_original[col].quantile(0.75)
                iqr = q3 - q1
                upper_bound = q3 + 5 * iqr
                lower_bound = q1 - 5 * iqr

                extreme_vals = ((df_original[col] > upper_bound) | (df_original[col] < lower_bound)).sum()
                if extreme_vals > 0 and len(cols_to_plot) < max_cols:
                    cols_to_plot.append(col)

        # Fill remaining slots with random columns
        remaining_cols = [c for c in columns if c not in cols_to_plot and df_original[c].dtype.kind in 'ifc']
        if remaining_cols and len(cols_to_plot) < max_cols:
            np.random.seed(42)  # For reproducibility
            sample_cols = np.random.choice(remaining_cols,
                                          min(max_cols - len(cols_to_plot), len(remaining_cols)),
                                          replace=False)
            cols_to_plot.extend(sample_cols)
    else:
        cols_to_plot = [c for c in columns if df_original[c].dtype.kind in 'ifc'][:max_cols]

    if not cols_to_plot:
        print("No suitable numeric columns for visualization.")
        return

    # Create plots
    fig, axes = plt.subplots(len(cols_to_plot), 2, figsize=(12, 4 * len(cols_to_plot)))

    for i, col in enumerate(cols_to_plot):
        # For single column case
        if len(cols_to_plot) == 1:
            ax1, ax2 = axes
        else:
            ax1, ax2 = axes[i]

        # Before cleaning - drop infinities for visualization
        before_data = df_original[col].replace([np.inf, -np.inf], np.nan).dropna()
        if len(before_data) > 0:
            sns.histplot(before_data, ax=ax1, bins=30, kde=True)
            ax1.set_title(f'Before Cleaning: {col}')
            ax1.set_ylabel('Count')

            # Add text with statistics
            stats_text = f"Mean: {before_data.mean():.2f}\n"
            stats_text += f"Std: {before_data.std():.2f}\n"
            stats_text += f"Min: {before_data.min():.2f}\n"
            stats_text += f"Max: {before_data.max():.2f}"
            ax1.text(0.05, 0.95, stats_text, transform=ax1.transAxes,
                    verticalalignment='top', bbox=dict(boxstyle='round', alpha=0.5))
        else:
            ax1.text(0.5, 0.5, 'No valid data before cleaning',
                    ha='center', va='center', transform=ax1.transAxes)

        # After cleaning
        after_data = df_cleaned[col].dropna()
        if len(after_data) > 0:
            sns.histplot(after_data, ax=ax2, bins=30, kde=True)
            ax2.set_title(f'After Cleaning: {col}')

            # Add text with statistics
            stats_text = f"Mean: {after_data.mean():.2f}\n"
            stats_text += f"Std: {after_data.std():.2f}\n"
            stats_text += f"Min: {after_data.min():.2f}\n"
            stats_text += f"Max: {after_data.max():.2f}"
            ax2.text(0.05, 0.95, stats_text, transform=ax2.transAxes,
                    verticalalignment='top', bbox=dict(boxstyle='round', alpha=0.5))
        else:
            ax2.text(0.5, 0.5, 'No valid data after cleaning',
                    ha='center', va='center', transform=ax2.transAxes)

    plt.tight_layout()
    plt.show()

    return

# Usage example:
# Assuming X_train is your dataset before cleaning
# Replace 'column_names' with actual column names if you want to focus on specific columns

# To analyze the entire dataset:
# cleaned_data = examine_dataset_cleaning(X_train)

# To analyze specific columns:
# cleaned_data = examine_dataset_cleaning(X_train, column_subset=['feature1', 'feature2'])

# Usage with your datasets:
# Before applying cleaning (using X_train from your original code)
print("Examining training data:")
cleaned_X_train = examine_dataset_cleaning(X_train)

# You can do the same for validation and test sets if needed
# print("\nExamining validation data:")
# cleaned_X_val = examine_dataset_cleaning(X_val)
#
# print("\nExamining test data:")
# cleaned_X_test = examine_dataset_cleaning(X_test)

# Modeling Algorithms

#XGBoost Model

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

xgb_params = {
    'n_estimators': 300,















































































































































































































































































































































































































































    'max_depth': 6,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0.1,
    'random_state': 42
}

# Hospital bed demand model
xgb_hosp = XGBRegressor(**xgb_params)
xgb_hosp.fit(X_train, y_train_hosp)
xgb_hosp_pred = xgb_hosp.predict(X_test)

# ICU demand model
xgb_icu = XGBRegressor(**xgb_params)
xgb_icu.fit(X_train, y_train_icu)
xgb_icu_pred = xgb_icu.predict(X_test)

# Evaluation
print("XGBoost Hospital Bed Demand Prediction:")
print(f"MAE: {mean_absolute_error(y_test_hosp, xgb_hosp_pred):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_hosp, xgb_hosp_pred)):.2f}")

print("\nXGBoost ICU Demand Prediction:")
print(f"MAE: {mean_absolute_error(y_test_icu, xgb_icu_pred):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_icu, xgb_icu_pred)):.2f}")

# Hyperparameter Tuning:

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 300, 500]
}
grid_search = GridSearchCV(XGBRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train_icu)
print("Best parameters:", grid_search.best_params_)

# Prophet Model

In [ ]:
from prophet import Prophet

# Prepare data for Prophet
prophet_df = df[df['location'] == 'United States'][['date', 'hosp_patients_per_million']].dropna()
prophet_df.columns = ['ds', 'y']

# Fit model
prophet_model = Prophet(seasonality_mode='multiplicative',
                       yearly_seasonality=True,
                       weekly_seasonality=True)
prophet_model.fit(prophet_df)

# Make future dataframe
future = prophet_model.make_future_dataframe(periods=14)
forecast = prophet_model.predict(future)

# Plot results
fig = prophet_model.plot(forecast)
plt.title('Prophet Forecast for Hospital Bed Demand')
plt.show()

In [ ]:
from prophet import Prophet
import matplotlib.pyplot as plt

# Prepare data for Prophet (US hospitalizations)
prophet_df = df[df['location'] == 'United States'][['date', 'hosp_patients_per_million']].dropna()
prophet_df.columns = ['ds', 'y']

# Fit model with tuned parameters
prophet_model = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.05,  # Adjusts trend flexibility
    interval_width=0.95            # 95% confidence intervals
)
prophet_model.fit(prophet_df)

# Generate 14-day forecast
future = prophet_model.make_future_dataframe(periods=14)
forecast = prophet_model.predict(future)

# Plot with professional formatting
fig = prophet_model.plot(forecast, xlabel='Date', ylabel='Hospitalized Patients per Million')
plt.title('Prophet Forecast: U.S. COVID-19 Hospital Bed Demand (2020-2024)', fontsize=14, pad=20)

# Enhance plot aesthetics
ax = plt.gca()
ax.set_facecolor('#f5f5f5')  # Light gray background
fig.patch.set_facecolor('white')  # White figure background

# Highlight key pandemic phases
plt.axvspan('2021-06-01', '2021-10-01', color='red', alpha=0.1, label='Delta Variant Dominance')
plt.axvspan('2021-12-01', '2022-03-01', color='purple', alpha=0.1, label='Omicron Wave')

plt.legend(loc='upper left', frameon=True)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# =============================================
# 1. DATA PREPARATION WITH EXTERNAL REGRESSORS
# =============================================

# Load and prepare US data with vaccination rates
us_data = df[df['location'] == 'United States'][[
    'date',
    'hosp_patients_per_million',
    'people_fully_vaccinated_per_hundred',
    'stringency_index'
]].dropna().copy()
us_data.columns = ['ds', 'y', 'vax_rate', 'policy_strictness']

# Create 7-day rolling features
us_data['case_growth'] = us_data['y'].pct_change(7)
us_data['vax_impact'] = us_data['vax_rate'] * us_data['policy_strictness'] / 100

# =============================================
# 2. ENHANCED PROPHET MODEL
# =============================================

# Initialize model with custom configurations
prophet_model = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.05,
    interval_width=0.95
)

# Add external regressors
prophet_model.add_regressor('vax_rate')
prophet_model.add_regressor('policy_strictness')
prophet_model.add_regressor('vax_impact')

# Add custom monthly seasonality
prophet_model.add_seasonality(
    name='monthly',
    period=30.5,
    fourier_order=5
)

# Fit model
prophet_model.fit(us_data)

# Create future dataframe with regressors
future = prophet_model.make_future_dataframe(periods=14)

# Propagate regressor values (using last known values for future)
for col in ['vax_rate', 'policy_strictness', 'vax_impact']:
    future[col] = us_data[col].iloc[-1]

# Generate forecast
forecast = prophet_model.predict(future)

# =============================================
# 3. COMPARISON WITH XGBOOST
# =============================================

# Prepare data for XGBoost
X = us_data[['vax_rate', 'policy_strictness', 'case_growth']]
y = us_data['y']

# Temporal train-test split (last 14 days for test)
X_train, X_test = X.iloc[:-14], X.iloc[-14:]
y_train, y_test = y.iloc[:-14], y.iloc[-14:]

# Train XGBoost
xgb_model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1
)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

# =============================================
# 4. VISUALIZATION
# =============================================

plt.figure(figsize=(14, 8))

# Prophet forecast
plt.plot(us_data['ds'], us_data['y'], 'k.', label='Observed Data')
plt.plot(forecast['ds'], forecast['yhat'], 'b-', linewidth=2, label='Prophet Forecast')
plt.fill_between(
    forecast['ds'],
    forecast['yhat_lower'],
    forecast['yhat_upper'],
    color='blue',
    alpha=0.1,
    label='95% CI'
)

# XGBoost predictions
plt.plot(X_test.index, xgb_pred, 'r--', linewidth=2, label='XGBoost Prediction')

# Formatting
plt.title('Comparative Forecast: Prophet vs XGBoost\nUS Hospital Bed Demand (Patients per Million)',
          fontsize=14, pad=20)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Hospitalized Patients per Million', fontsize=12)
plt.legend(loc='upper left')
plt.grid(alpha=0.3)
plt.tight_layout()

# Annotate key events
plt.axvspan(pd.to_datetime('2021-06-01'), pd.to_datetime('2021-10-01'),
            color='red', alpha=0.1, label='Delta Variant')
plt.axvspan(pd.to_datetime('2021-12-01'), pd.to_datetime('2022-02-01'),
            color='purple', alpha=0.1, label='Omicron Wave')

plt.show()

# =============================================
# 5. MODEL EVALUATION
# =============================================

# Prophet evaluation on test period
prophet_test_pred = forecast.iloc[-14:]['yhat']
prophet_mae = mean_absolute_error(y_test, prophet_test_pred)

# Print results
print(f"\nModel Performance (Last 14 Days):")
print(f"Prophet MAE: {prophet_mae:.2f} patients/million")
print(f"XGBoost MAE: {mean_absolute_error(y_test, xgb_pred):.2f} patients/million")
print(f"Baseline (Last Value) MAE: {mean_absolute_error(y_test, [y_train.iloc[-1]]*14):.2f} patients/million")

# Feature importance
print("\nXGBoost Feature Importance:")
for feat, imp in zip(X.columns, xgb_model.feature_importances_):
    print(f"{feat}: {imp:.3f}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from prophet import Prophet
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

## =============================================
## 1. DATA PREPARATION
## =============================================

# Load and prepare US data
us_data = df[df['location'] == 'United States'][[
    'date', 'hosp_patients_per_million',
    'people_fully_vaccinated_per_hundred',
    'stringency_index',
    'new_cases_per_million'
]].dropna().copy()

# Create lagged features with forward filling
us_data['cases_lagged_7'] = us_data['new_cases_per_million'].shift(7).ffill()
us_data['cases_lagged_14'] = us_data['new_cases_per_million'].shift(14).ffill()
us_data['hosp_lagged_7'] = us_data['hosp_patients_per_million'].shift(7).ffill()

# Calculate moving averages (minimum 1 period)
us_data['cases_7day_avg'] = us_data['new_cases_per_million'].rolling(7, min_periods=1).mean()
us_data['vax_policy_interaction'] = (us_data['people_fully_vaccinated_per_hundred'] *
                                   us_data['stringency_index'] / 100)

# Ensure no NaN values remain
us_data = us_data.dropna()

## =============================================
## 2. PROPHET MODEL
## =============================================

# Initialize and configure Prophet
prophet_model = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    changepoint_prior_scale=0.03,
    interval_width=0.90
)

# Add regressors
for col in ['people_fully_vaccinated_per_hundred', 'stringency_index',
           'vax_policy_interaction', 'cases_7day_avg']:
    prophet_model.add_regressor(col)

# Prepare Prophet dataframe
prophet_df = us_data.rename(columns={'date': 'ds', 'hosp_patients_per_million': 'y'})
prophet_df = prophet_df[['ds', 'y'] +
                       ['people_fully_vaccinated_per_hundred', 'stringency_index',
                        'vax_policy_interaction', 'cases_7day_avg']]

# Fit model
prophet_model.fit(prophet_df)

# Create future dataframe
future = prophet_model.make_future_dataframe(periods=14)
for col in ['people_fully_vaccinated_per_hundred', 'stringency_index',
           'vax_policy_interaction', 'cases_7day_avg']:
    future[col] = us_data[col].iloc[-1]

# Generate forecast
forecast = prophet_model.predict(future)

## =============================================
## 3. XGBOOST MODEL
## =============================================

# Prepare features
features = ['people_fully_vaccinated_per_hundred', 'stringency_index',
            'cases_lagged_7', 'cases_lagged_14', 'hosp_lagged_7']
X = us_data[features]
y = us_data['hosp_patients_per_million']

# Temporal train-test split
test_size = 14
X_train, X_test = X.iloc[:-test_size], X.iloc[-test_size:]
y_train, y_test = y.iloc[:-test_size], y.iloc[-test_size:]

# Train XGBoost
xgb_model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0.1,
    random_state=42
)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

## =============================================
## 4. VISUALIZATION (WITH PROPER STYLING)
## =============================================

plt.figure(figsize=(16, 8))

# Use available matplotlib style
plt.style.use('ggplot')  # Alternative: 'seaborn', 'fivethirtyeight', 'bmh'

# Plot observed data
plt.plot(us_data['date'], us_data['hosp_patients_per_million'],
         'o', markersize=4, color='#333333', alpha=0.6, label='Observed Data')

# Plot Prophet forecast
plt.plot(forecast['ds'], forecast['yhat'],
         color='#1f77b4', linewidth=2.5, label='Prophet Forecast')
plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'],
                 color='#1f77b4', alpha=0.15, label='90% CI')

# Plot XGBoost predictions
plt.plot(X_test.index, xgb_pred,
         '--', color='#d62728', linewidth=2.5, label='XGBoost Prediction')

# Formatting
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.xticks(rotation=45)
plt.ylabel('Hospitalized Patients per Million', fontsize=12)
plt.xlabel('Date', fontsize=12)
plt.title('Comparative Forecast of US Hospital Bed Demand',
          fontsize=14, pad=20)

# Add performance annotations
plt.annotate(f'Prophet MAE: {mean_absolute_error(y_test, forecast.iloc[-test_size:]["yhat"]):.1f}',
             xy=(0.72, 0.15), xycoords='axes fraction', color='#1f77b4')
plt.annotate(f'XGBoost MAE: {mean_absolute_error(y_test, xgb_pred):.1f}',
             xy=(0.72, 0.10), xycoords='axes fraction', color='#d62728')

# Final touches
plt.legend(loc='upper left', frameon=True, facecolor='white')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## =============================================
## 5. MODEL EVALUATION
## =============================================

print("\nModel Performance Comparison (Last 14 Days):")
print(f"Prophet MAE: {mean_absolute_error(y_test, forecast.iloc[-test_size:]['yhat']):.2f}")
print(f"XGBoost MAE: {mean_absolute_error(y_test, xgb_pred):.2f}")

print("\nXGBoost Feature Importance:")
for feat, imp in zip(features, xgb_model.feature_importances_):
    print(f"{feat}: {imp:.3f}")

In [ ]:
# Prophet Implementation and Evaluation
plt.figure(figsize=(12, 6))

# Plot components
fig = prophet_model.plot_components(forecast)
plt.suptitle('Prophet Model Components Analysis', y=1.02)
plt.tight_layout()

# Forecast plot
plt.figure(figsize=(12, 6))
plt.plot(us_data['date'], us_data['hosp_patients_per_million'], 'o', color='#2c3e50', label='Observed')
plt.plot(forecast['ds'], forecast['yhat'], color='#3498db', label='Prophet Forecast')
plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'],
                 color='#3498db', alpha=0.1)
plt.title('Prophet Forecast with Uncertainty', pad=20)
plt.ylabel('Patients per Million')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()

print(f"Prophet MAE: {mean_absolute_error(y_test, forecast.iloc[-test_size:]['yhat']):.2f}")

# LSTM MODEL

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.metrics import mean_absolute_error

## =============================================
## 1. DATA PREPARATION
## =============================================

# Prepare data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(us_data[['hosp_patients_per_million']])

# Create sequences with date tracking
def create_sequences(data, dates, n_steps):
    X, y, y_dates = [], [], []
    for i in range(len(data)-n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps])
        y_dates.append(dates.iloc[i+n_steps])  # Capture corresponding dates
    return np.array(X), np.array(y), np.array(y_dates)

n_steps = 14
X, y, y_dates = create_sequences(scaled_data, us_data['date'], n_steps)

# Split data
test_size = 14
X_train, X_test = X[:-test_size], X[-test_size:]
y_train, y_test = y[:-test_size], y[-test_size:]
test_dates = y_dates[-test_size:]  # Keep test dates for plotting

## =============================================
## 2. LSTM MODEL IMPLEMENTATION
## =============================================

# Build model with proper input layer
model = Sequential([
    Input(shape=(n_steps, 1)),  # Proper input specification
    LSTM(50, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mae')

# Train
history = model.fit(X_train, y_train, epochs=50, verbose=0, validation_split=0.2)

# Predict
lstm_pred = model.predict(X_test)
lstm_pred = scaler.inverse_transform(lstm_pred).flatten()
y_test_actual = scaler.inverse_transform(y_test).flatten()

## =============================================
## 3. VISUALIZATION
## =============================================

plt.figure(figsize=(14, 7))

# Manual styling for consistency
plt.rcParams.update({
    'axes.facecolor': 'white',
    'grid.color': '#e0e0e0',
    'axes.edgecolor': '#404040'
})

# Plot actual vs predicted
plt.plot(test_dates, y_test_actual, 'o-', color='#2c3e50',
         label='Actual Demand', linewidth=2, markersize=8)
plt.plot(test_dates, lstm_pred, 's--', color='#9b59b6',
         label='LSTM Prediction', markersize=8, linewidth=2)

# Formatting
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.xticks(rotation=45)

plt.title('LSTM Model Performance: Hospital Bed Demand Prediction', pad=20)
plt.ylabel('Patients per Million')
plt.xlabel('Date')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()

# Annotate performance
plt.annotate(f'MAE: {mean_absolute_error(y_test_actual, lstm_pred):.2f}',
             xy=(0.75, 0.9), xycoords='axes fraction', color='#9b59b6',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

## =============================================
## 4. MODEL EVALUATION
## =============================================

print("\nLSTM Model Evaluation:")
print(f"Test MAE: {mean_absolute_error(y_test_actual, lstm_pred):.2f} patients/million")

# Plot training history
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('LSTM Training History')
plt.ylabel('MAE Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
!pip install optuna pytorch-forecasting
import optuna
import torch
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, TimeDistributed
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer

## =============================================
## 1. HYBRID MODEL ARCHITECTURES
## =============================================

### A. CNN-LSTM Hybrid
def build_cnn_lstm(hp):
    model = Sequential([
        Input(shape=(n_steps, len(features_to_scale))),
        Conv1D(filters=hp.Int('filters', 32, 128, step=32),
               kernel_size=hp.Int('kernel_size', 2, 5),
               activation='relu'),
        MaxPooling1D(pool_size=2),
        LSTM(hp.Int('lstm_units', 32, 128, step=32),
              return_sequences=True),
        Dropout(hp.Float('dropout', 0.1, 0.5)),
        LSTM(hp.Int('lstm_units_2', 16, 64, step=16)),
        Dense(1)
    ])
    model.compile(optimizer=Adam(hp.Float('lr', 1e-4, 1e-2, sampling='log')),
                loss='mae')
    return model

### B. Transformer-LSTM Hybrid
class TransformerLSTM(Model):
    def __init__(self, num_heads=4, key_dim=64, ff_dim=128):
        super().__init__()
        self.lstm = LSTM(64, return_sequences=True)
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)
        self.norm = LayerNormalization()
        self.ffn = Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(64)
        ])
        self.final_dense = Dense(1)

    def call(self, inputs):
        x = self.lstm(inputs)
        attn_output = self.attention(x, x)
        x = self.norm(x + attn_output)
        x = self.ffn(x)
        return self.final_dense(x[:, -1, :])

### C. Bayesian-optimized GRU
def gru_objective(trial):
    params = {
        'units1': trial.suggest_int('units1', 32, 128),
        'units2': trial.suggest_int('units2', 16, 64),
        'dropout': trial.suggest_float('dropout', 0.1, 0.5),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    }

    model = Sequential([
        GRU(params['units1'], return_sequences=True),
        Dropout(params['dropout']),
        GRU(params['units2']),
        Dense(1)
    ])
    model.compile(optimizer=Adam(params['lr']), loss='mae')

    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=50,
        verbose=0
    )
    return min(history.history['val_loss'])

### D. Temporal Fusion Transformer
def prepare_tft_data():
    dataset = TimeSeriesDataSet(
        us_data.reset_index(),
        time_idx="time_idx",
        target="hosp_patients_per_million",
        group_ids=["group"],
        min_encoder_length=max_encoder_length // 2,
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=max_prediction_length,
        time_varying_known_reals=["time_idx"] + features_to_scale,
        target_normalizer=GroupNormalizer(groups=["group"]),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
    )
    return dataset

## =============================================
## 2. TRAINING FRAMEWORK
## =============================================

# Bayesian Optimization for GRU
gru_study = optuna.create_study(direction='minimize')
gru_study.optimize(gru_objective, n_trials=30)

# Build best GRU model
best_gru = Sequential([
    GRU(gru_study.best_params['units1'], return_sequences=True),
    Dropout(gru_study.best_params['dropout']),
    GRU(gru_study.best_params['units2']),
    Dense(1)
])
best_gru.compile(optimizer=Adam(gru_study.best_params['lr']), loss='mae')

# Hybrid Models
models = {
    'CNN-LSTM': build_cnn_lstm(optuna.FixedTrial({
        'filters': 64,
        'kernel_size': 3,
        'lstm_units': 64,
        'dropout': 0.2,
        'lr': 0.001
    })),
    'Transformer-LSTM': TransformerLSTM(),
    'Bayesian GRU': best_gru
}

# Train all models
histories = {}
predictions = {}
for name, model in models.items():
    print(f"\nTraining {name}...")
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        verbose=1
    )
    histories[name] = history.history
    predictions[name] = model.predict(X_test).flatten()

## =============================================
## 3. COMPREHENSIVE EVALUATION
## =============================================

plt.figure(figsize=(18, 9))

# Plot actual vs predictions
colors = ['#3498db', '#e74c3c', '#2ecc71', '#9b59b6']
plt.plot(test_dates, y_test_actual, 'o-', color='#2c3e50',
         linewidth=3, markersize=8, label='Actual')

for i, (name, preds) in enumerate(predictions.items()):
    plt.plot(test_dates, preds, '--', color=colors[i],
             linewidth=2.5, label=f'{name} (MAE: {mean_absolute_error(y_test_actual, preds):.2f})')

# Formatting
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.xticks(rotation=45)

plt.title('Advanced Model Comparison: Hospital Bed Demand Prediction',
          pad=20, fontsize=16)
plt.ylabel('Patients per Million', fontsize=14)
plt.xlabel('Date', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.4)
plt.legend(fontsize=12)

# Performance table
mae_scores = {name: mean_absolute_error(y_test_actual, preds)
              for name, preds in predictions.items()}
sorted_models = sorted(mae_scores.items(), key=lambda x: x[1])

table_data = [[name, f"{mae:.2f}"] for name, mae in sorted_models]
table = plt.table(cellText=table_data,
                 colLabels=['Model', 'MAE'],
                 loc='lower right',
                 cellLoc='center',
                 bbox=[0.7, 0.15, 0.25, 0.3])
table.auto_set_font_size(False)
table.set_fontsize(12)

plt.tight_layout()
plt.show()

## =============================================
## 4. MODEL ANALYSIS
## =============================================

# Training dynamics
plt.figure(figsize=(16, 6))
for i, (name, history) in enumerate(histories.items()):
    plt.plot(history['loss'], '--', color=colors[i], alpha=0.7, label=f'{name} Train')
    plt.plot(history['val_loss'], '-', color=colors[i], label=f'{name} Val')

plt.title('Training Dynamics Comparison', pad=20, fontsize=16)
plt.ylabel('MAE Loss', fontsize=14)
plt.xlabel('Epoch', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.4)
plt.legend(fontsize=12)
plt.show()

# Feature importance for TFT
tft_dataset = prepare_tft_data()
tft = TemporalFusionTransformer.from_dataset(
    tft_dataset,
    learning_rate=0.03,
    hidden_size=32,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=16,
    output_size=1,
    loss=torch.nn.MSELoss()
)

# (Note: TFT training would require PyTorch DataLoader setup)

# Temporal Fusion Transformer (Advanced Model)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

# Load the dataset (replace with your actual path)
data_path = '/content/drive/MyDrive/Medical_Resource_Prediction/owid-covid-data.csv'
us_data = pd.read_csv(data_path, parse_dates=['date'])

# Filter for United States data only
us_data = us_data[us_data['location'] == 'United States'].set_index('date')

## =============================================
## 1. DATA PREPARATION FOR RANDOM FOREST
## =============================================

# Select features and target
features = ['new_cases_per_million',
            'people_fully_vaccinated_per_hundred',
            'stringency_index',
            'hospital_beds_per_thousand']
target = 'hosp_patients_per_million'

# Prepare data (ensure no missing values)
model_data = us_data[features + [target]].dropna()

# Create temporal features
model_data['day_of_week'] = model_data.index.dayofweek
model_data['month'] = model_data.index.month

# Create lagged features (7-day lags)
model_data['cases_lag7'] = model_data['new_cases_per_million'].shift(7)
model_data['hosp_lag7'] = model_data[target].shift(7)

# Remove rows with NA values from lagging
model_data = model_data.dropna()

# Final feature set
features_extended = features + ['day_of_week', 'month', 'cases_lag7', 'hosp_lag7']

## =============================================
## 2. TRAIN-TEST SPLIT
## =============================================

# Temporal split (last 14 days for test)
test_size = 14
X_train = model_data[features_extended][:-test_size]
y_train = model_data[target][:-test_size]
X_test = model_data[features_extended][-test_size:]
y_test = model_data[target][-test_size:]
test_dates = model_data.index[-test_size:]

## =============================================
## 3. RANDOM FOREST IMPLEMENTATION
## =============================================

# Train Random Forest
rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1  # Use all available cores
)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

## =============================================
## 4. VISUALIZATION
## =============================================

plt.figure(figsize=(14, 7))

# Plot actual vs predicted
plt.plot(test_dates, y_test, 'o-', color='#2c3e50',
         linewidth=2, markersize=8, label='Actual Demand')
plt.plot(test_dates, rf_pred, 's--', color='#2ecc71',
         linewidth=2, markersize=8, label='RF Prediction')

# Formatting
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.xticks(rotation=45)

plt.title('Random Forest: Hospital Bed Demand Prediction\nUnited States', pad=20)
plt.ylabel('Patients per Million')
plt.xlabel('Date')
plt.grid(True, linestyle='--', alpha=0.4)
plt.legend()

# Annotate performance
plt.annotate(f'MAE: {mean_absolute_error(y_test, rf_pred):.2f}',
             xy=(0.75, 0.9), xycoords='axes fraction',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

## =============================================
## 5. FEATURE IMPORTANCE ANALYSIS
## =============================================

plt.figure(figsize=(10, 6))
importances = rf_model.feature_importances_
sorted_idx = np.argsort(importances)[-10:]  # Top 10 features

plt.barh(range(len(sorted_idx)), importances[sorted_idx], color='#27ae60')
plt.yticks(range(len(sorted_idx)), np.array(features_extended)[sorted_idx])
plt.title('Random Forest Feature Importance')
plt.xlabel('Importance Score')
plt.grid(True, linestyle='--', alpha=0.4, axis='x')
plt.tight_layout()
plt.show()

print("\nTop Predictive Features:")
for feature, importance in zip(np.array(features_extended)[sorted_idx],
                             importances[sorted_idx]):
    print(f"{feature}: {importance:.3f}")

# Advanced Hybrid Architectures with Bayesian Optimization

In [ ]:
!pip install optuna pytorch-forecasting
import optuna
import torch
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, TimeDistributed
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer

## =============================================
## 1. HYBRID MODEL ARCHITECTURES
## =============================================

### A. CNN-LSTM Hybrid
def build_cnn_lstm(hp):
    model = Sequential([
        Input(shape=(n_steps, len(features_to_scale))),
        Conv1D(filters=hp.Int('filters', 32, 128, step=32),
               kernel_size=hp.Int('kernel_size', 2, 5),
               activation='relu'),
        MaxPooling1D(pool_size=2),
        LSTM(hp.Int('lstm_units', 32, 128, step=32),
              return_sequences=True),
        Dropout(hp.Float('dropout', 0.1, 0.5)),
        LSTM(hp.Int('lstm_units_2', 16, 64, step=16)),
        Dense(1)
    ])
    model.compile(optimizer=Adam(hp.Float('lr', 1e-4, 1e-2, sampling='log')),
                loss='mae')
    return model

### B. Transformer-LSTM Hybrid
class TransformerLSTM(Model):
    def __init__(self, num_heads=4, key_dim=64, ff_dim=128):
        super().__init__()
        self.lstm = LSTM(64, return_sequences=True)
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)
        self.norm = LayerNormalization()
        self.ffn = Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(64)
        ])
        self.final_dense = Dense(1)

    def call(self, inputs):
        x = self.lstm(inputs)
        attn_output = self.attention(x, x)
        x = self.norm(x + attn_output)
        x = self.ffn(x)
        return self.final_dense(x[:, -1, :])

### C. Bayesian-optimized GRU
def gru_objective(trial):
    params = {
        'units1': trial.suggest_int('units1', 32, 128),
        'units2': trial.suggest_int('units2', 16, 64),
        'dropout': trial.suggest_float('dropout', 0.1, 0.5),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    }

    model = Sequential([
        GRU(params['units1'], return_sequences=True),
        Dropout(params['dropout']),
        GRU(params['units2']),
        Dense(1)
    ])
    model.compile(optimizer=Adam(params['lr']), loss='mae')

    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=50,
        verbose=0
    )
    return min(history.history['val_loss'])

### D. Temporal Fusion Transformer
def prepare_tft_data():
    dataset = TimeSeriesDataSet(
        us_data.reset_index(),
        time_idx="time_idx",
        target="hosp_patients_per_million",
        group_ids=["group"],
        min_encoder_length=max_encoder_length // 2,
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=max_prediction_length,
        time_varying_known_reals=["time_idx"] + features_to_scale,
        target_normalizer=GroupNormalizer(groups=["group"]),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
    )
    return dataset

## =============================================
## 2. TRAINING FRAMEWORK
## =============================================

# Bayesian Optimization for GRU
gru_study = optuna.create_study(direction='minimize')
gru_study.optimize(gru_objective, n_trials=30)

# Build best GRU model
best_gru = Sequential([
    GRU(gru_study.best_params['units1'], return_sequences=True),
    Dropout(gru_study.best_params['dropout']),
    GRU(gru_study.best_params['units2']),
    Dense(1)
])
best_gru.compile(optimizer=Adam(gru_study.best_params['lr']), loss='mae')

# Hybrid Models
models = {
    'CNN-LSTM': build_cnn_lstm(optuna.FixedTrial({
        'filters': 64,
        'kernel_size': 3,
        'lstm_units': 64,
        'dropout': 0.2,
        'lr': 0.001
    })),
    'Transformer-LSTM': TransformerLSTM(),
    'Bayesian GRU': best_gru
}

# Train all models
histories = {}
predictions = {}
for name, model in models.items():
    print(f"\nTraining {name}...")
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        verbose=1
    )
    histories[name] = history.history
    predictions[name] = model.predict(X_test).flatten()

## =============================================
## 3. COMPREHENSIVE EVALUATION
## =============================================

plt.figure(figsize=(18, 9))

# Plot actual vs predictions
colors = ['#3498db', '#e74c3c', '#2ecc71', '#9b59b6']
plt.plot(test_dates, y_test_actual, 'o-', color='#2c3e50',
         linewidth=3, markersize=8, label='Actual')

for i, (name, preds) in enumerate(predictions.items()):
    plt.plot(test_dates, preds, '--', color=colors[i],
             linewidth=2.5, label=f'{name} (MAE: {mean_absolute_error(y_test_actual, preds):.2f})')

# Formatting
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.xticks(rotation=45)

plt.title('Advanced Model Comparison: Hospital Bed Demand Prediction',
          pad=20, fontsize=16)
plt.ylabel('Patients per Million', fontsize=14)
plt.xlabel('Date', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.4)
plt.legend(fontsize=12)

# Performance table
mae_scores = {name: mean_absolute_error(y_test_actual, preds)
              for name, preds in predictions.items()}
sorted_models = sorted(mae_scores.items(), key=lambda x: x[1])

table_data = [[name, f"{mae:.2f}"] for name, mae in sorted_models]
table = plt.table(cellText=table_data,
                 colLabels=['Model', 'MAE'],
                 loc='lower right',
                 cellLoc='center',
                 bbox=[0.7, 0.15, 0.25, 0.3])
table.auto_set_font_size(False)
table.set_fontsize(12)

plt.tight_layout()
plt.show()

## =============================================
## 4. MODEL ANALYSIS
## =============================================

# Training dynamics
plt.figure(figsize=(16, 6))
for i, (name, history) in enumerate(histories.items()):
    plt.plot(history['loss'], '--', color=colors[i], alpha=0.7, label=f'{name} Train')
    plt.plot(history['val_loss'], '-', color=colors[i], label=f'{name} Val')

plt.title('Training Dynamics Comparison', pad=20, fontsize=16)
plt.ylabel('MAE Loss', fontsize=14)
plt.xlabel('Epoch', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.4)
plt.legend(fontsize=12)
plt.show()

# Feature importance for TFT
tft_dataset = prepare_tft_data()
tft = TemporalFusionTransformer.from_dataset(
    tft_dataset,
    learning_rate=0.03,
    hidden_size=32,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=16,
    output_size=1,
    loss=torch.nn.MSELoss()
)

# (Note: TFT training would require PyTorch DataLoader setup)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf  # Added this import
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, GRU, Input, Layer
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

## =============================================
## 1. DATA PREPARATION (Enhanced)
## =============================================

# Prepare scaled data with more features
scaler = MinMaxScaler()
features_to_scale = ['hosp_patients_per_million', 'new_cases_per_million',
                    'people_fully_vaccinated_per_hundred']
scaled_data = scaler.fit_transform(us_data[features_to_scale])

# Create sequences with multiple features
def create_sequences(data, dates, n_steps):
    X, y, y_dates = [], [], []
    for i in range(len(data)-n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps, 0])  # Predict only hospitalization
        y_dates.append(dates.iloc[i+n_steps])
    return np.array(X), np.array(y), np.array(y_dates)

n_steps = 14
X, y, y_dates = create_sequences(scaled_data, us_data['date'], n_steps)

# Train-test split
test_size = 14
X_train, X_test = X[:-test_size], X[-test_size:]
y_train, y_test = y[:-test_size], y[-test_size:]
test_dates = y_dates[-test_size:]

## =============================================
## 2. MODEL ARCHITECTURES
## =============================================

### A. Stacked LSTM Model
lstm_model = Sequential([
    Input(shape=(n_steps, len(features_to_scale))),
    LSTM(64, activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])
lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')

### B. LSTM with Attention Mechanism
class AttentionLayer(Layer):
    def __init__(self, units=32):
        super(AttentionLayer, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, inputs):
        hidden_states = inputs
        score = self.V(tf.nn.tanh(self.W1(hidden_states) + self.W2(hidden_states)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * hidden_states
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[2])

# Build model
inputs = Input(shape=(n_steps, len(features_to_scale)))
lstm_out = LSTM(64, activation='relu', return_sequences=True)(inputs)
attention = AttentionLayer(32)(lstm_out)
outputs = Dense(1)(attention)
attention_model = Model(inputs, outputs)
attention_model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')

### C. GRU Model
gru_model = Sequential([
    Input(shape=(n_steps, len(features_to_scale))),
    GRU(64, activation='relu', return_sequences=True),
    Dropout(0.2),
    GRU(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])
gru_model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')

## =============================================
## 3. TRAINING AND EVALUATION
## =============================================

models = {
    'Stacked LSTM': lstm_model,
    'LSTM with Attention': attention_model,
    'GRU': gru_model
}

history_dict = {}
predictions = {}

# Train and evaluate all models
for name, model in models.items():
    print(f"\nTraining {name}...")
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        verbose=0
    )
    history_dict[name] = history.history
    predictions[name] = model.predict(X_test).flatten()

# Inverse transform predictions
y_test_actual = scaler.inverse_transform(
    np.concatenate([y_test.reshape(-1,1), np.zeros((len(y_test), len(features_to_scale)-1))], axis=1)
)[:,0]

for name in predictions:
    preds = predictions[name]
    # Create dummy array for inverse transform
    dummy = np.zeros((len(preds), len(features_to_scale)))
    dummy[:,0] = preds
    predictions[name] = scaler.inverse_transform(dummy)[:,0]

## =============================================
## 4. VISUALIZATION AND COMPARISON
## =============================================

plt.figure(figsize=(16, 8))

# Plot actual values
plt.plot(test_dates, y_test_actual, 'o-', color='#2c3e50',
         linewidth=3, markersize=8, label='Actual Demand')

# Plot model predictions
colors = ['#3498db', '#e74c3c', '#2ecc71']
for i, (name, preds) in enumerate(predictions.items()):
    plt.plot(test_dates, preds, '--', color=colors[i],
             linewidth=2.5, label=f'{name} (MAE: {mean_absolute_error(y_test_actual, preds):.2f})')

# Formatting
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.xticks(rotation=45)

plt.title('Sequence Model Comparison: Hospital Bed Demand Prediction', pad=20, fontsize=14)
plt.ylabel('Patients per Million', fontsize=12)
plt.xlabel('Date', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.4)
plt.legend(fontsize=12)

# Add model performance table
model_mae = {name: mean_absolute_error(y_test_actual, preds)
             for name, preds in predictions.items()}
sorted_models = sorted(model_mae.items(), key=lambda x: x[1])

table_data = [[name, f"{mae:.2f}"] for name, mae in sorted_models]
table = plt.table(cellText=table_data,
                 colLabels=['Model', 'MAE'],
                 loc='lower right',
                 cellLoc='center',
                 bbox=[0.7, 0.15, 0.25, 0.3])
table.auto_set_font_size(False)
table.set_fontsize(12)

plt.tight_layout()
plt.show()

## =============================================
## 5. TRAINING HISTORY ANALYSIS
## =============================================

plt.figure(figsize=(14, 6))
for i, (name, history) in enumerate(history_dict.items()):
    plt.plot(history['loss'], '--', color=colors[i], alpha=0.7, label=f'{name} Training')
    plt.plot(history['val_loss'], '-', color=colors[i], label=f'{name} Validation')

plt.title('Model Training Histories', pad=20, fontsize=14)
plt.ylabel('MAE Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.4)
plt.legend(fontsize=10)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
file_path = '/content/drive/MyDrive/Medical_Resource_Prediction/owid-covid-data.csv'
df = pd.read_csv(file_path)

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"Time period covered: {df['date'].min()} to {df['date'].max()}")
print(f"Number of countries/regions: {df['location'].nunique()}")

# Display the first few rows
df.head()

# Summary statistics
df.describe()

# Check for missing values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_data = pd.DataFrame({'Missing Values': missing_values,
                             'Percentage': missing_percentage})
missing_data = missing_data[missing_data['Missing Values'] > 0].sort_values('Percentage', ascending=False)
missing_data

In [ ]:
from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='Missing Values', y='Percentage', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
_df_3['Missing Values'].plot(kind='line', figsize=(8, 4), title='Missing Values')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# @title Missing Values

from matplotlib import pyplot as plt
missing_data['Missing Values'].plot(kind='line', figsize=(8, 4), title='Missing Values')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
_df_0['Missing Values'].plot(kind='hist', bins=20, title='Missing Values')
plt.gca().spines[['top', 'right',]].set_visible(False)

# PROPHET MODEL

In [ ]:
from prophet import Prophet

# Load cleaned data
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Medical_Resource_Prediction/cleaned_data.csv')

# Prepare Prophet format (ds = timestamp, y = target variable)
prophet_data = data[['date', 'hospital_admissions']].rename(columns={'date': 'ds', 'hospital_admissions': 'y'})

# Initialize and fit model
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=10
)
model.add_country_holidays(country_name='US')  # Adjust based on region
model.fit(prophet_data)

# Make future dataframe and predict
future = model.make_future_dataframe(periods=30)  # 30-day forecast
forecast = model.predict(future)

# Plot components
fig = model.plot_components(forecast)

# Ensemble Approach
#Implementation Code:

In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_absolute_error

# Initialize individual models
prophet = Prophet(...)  # Configured as before
rf = RandomForestRegressor(...)  # Configured as before
xgb = xgb.XGBRegressor(...)  # Configured as before

# Create weighted ensemble
ensemble = VotingRegressor([
    ('prophet', prophet),
    ('rf', rf),
    ('xgb', xgb)],
    weights=[0.2, 0.3, 0.5]  # Adjust based on validation
)

# Dynamic weighting based on forecast horizon
def get_dynamic_weights(horizon):
    if horizon <= 7:   return [0.1, 0.2, 0.7]  # XGBoost heavy
    elif horizon <= 14: return [0.2, 0.3, 0.5]  # Balanced
    else:              return [0.5, 0.3, 0.2]  # Prophet heavy

# Evaluate
for horizon in [7, 14, 30]:
    weights = get_dynamic_weights(horizon)
    ensemble.set_params(weights=weights)
    ensemble.fit(X_train, y_train)
    preds = ensemble.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    print(f"MAE for {horizon}-day forecast: {mae:.2f}")

# LSTM Networks
# Implementation Code:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Prepare sequential data
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data)-n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps])
    return np.array(X), np.array(y)

n_steps = 14  # 2-week lookback
X, y = create_sequences(data.values, n_steps)

# Build model
model = Sequential([
    LSTM(100, activation='relu', input_shape=(n_steps, X.shape[2])),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train with early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10)
model.fit(
    X, y,
    epochs=100,
    validation_split=0.2,
    callbacks=[early_stop],
    batch_size=32,
    verbose=1
)

In [ ]:
# First, mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

# Import required libraries
import pandas as pd
import os

# Define the data path
data_path = '/content/drive/MyDrive/Medical_Resource_Prediction/cleaned_data.csv'

# Verify the file exists
if os.path.exists(data_path):
    try:
        # Load the data
        data = pd.read_csv(data_path)
        print("Data loaded successfully!")
        print(f"Data shape: {data.shape}")
        print("\nFirst 5 rows:")
        print(data.head())
    except Exception as e:
        print(f"Error loading file: {e}")
else:
    print(f"File not found at: {data_path}")
    print("\nAvailable files in /content/drive/MyDrive/:")
    print(os.listdir('/content/drive/MyDrive/'))

    # Check if Medical_Resource_Prediction directory exists
    if 'Medical_Resource_Prediction' in os.listdir('/content/drive/MyDrive/'):
        print("\nFiles in Medical_Resource_Prediction folder:")
        print(os.listdir('/content/drive/MyDrive/Medical_Resource_Prediction/'))
    else:
        print("\nMedical_Resource_Prediction folder not found in your Google Drive")

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesCV

xgb_params = {
    'n_estimators': 300,
    'max_depth': 6,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0.1,
    'random_state': 42
}

# Time-series cross-validation
tscv = TimeSeriesCV(n_splits=5)
xgb = XGBRegressor(**xgb_params)
xgb_scores = cross_val_score(xgb, train[features], train['future_icu_demand'],
                            cv=tscv, scoring='neg_mean_absolute_error')

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score

# ====================== 1. LOAD OR CREATE YOUR DATA ======================
# Example: Create synthetic time-series data (replace with your actual data)
dates = pd.date_range(start="2020-01-01", periods=100, freq="D")
train = pd.DataFrame({
    "date": dates,
    "feature1": np.random.rand(100) * 10,
    "feature2": np.random.rand(100) * 5,
    "future_icu_demand": np.random.rand(100) * 100  # Target variable
})

# Set 'date' as index (optional for time-series)
train.set_index("date", inplace=True)

# ====================== 2. DEFINE FEATURES & TARGET ======================
features = ["feature1", "feature2"]  # Predictors
target = "future_icu_demand"         # Target column

# ====================== 3. XGBOOST MODEL & TIME-SERIES CV ======================
xgb_params = {
    'n_estimators': 300,
    'max_depth': 6,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0.1,
    'random_state': 42
}

# Time-series cross-validation
tscv = TimeSeriesSplit(n_splits=5)
xgb = XGBRegressor(**xgb_params)

# Run cross-validation
xgb_scores = cross_val_score(
    xgb,
    train[features],          # Features (predictors)
    train[target],            # Target variable
    cv=tscv,
    scoring="neg_mean_absolute_error"
)

# Convert negative MAE to positive (sklearn convention)
mae_scores = -xgb_scores

print("Cross-validation MAE scores:", mae_scores)
print(f"Mean MAE: {mae_scores.mean():.2f}")